BigData. Фреймворк Apache Spark

Урок 2. Операции с данными: агрегаты, джойны. Оптимизация SQL-запросов

Используя данные из занятия необходимо ответить на вопросы:
1. Сколько строк в таблице?
2. Сколько строк, где пол указан мужской?
3. С помощью одной команды посчитать количество выживших мужчин и женщин.
4. С помощью одной команды определить средний возраст пассажиров каждого из Pclass`ов.

In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {
    "id": "8Zc4sdy0s35D"
   },
   "source": [
    "# Создаём точку входа в Spark"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "colab": {
     "base_uri": "https://localhost:8080/"
    },
    "id": "0yCimXIYGS76",
    "outputId": "2f679bcb-d9f5-4538-df40-c31045b7217f"
   },
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/\n",
      "Requirement already satisfied: pyspark in /usr/local/lib/python3.8/dist-packages (3.3.1)\n",
      "Requirement already satisfied: py4j==0.10.9.5 in /usr/local/lib/python3.8/dist-packages (from pyspark) (0.10.9.5)\n"
     ]
    }
   ],
   "source": [
    "!pip install pyspark"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "id": "idrMQH4dj605"
   },
   "outputs": [],
   "source": [
    "# Создаём спарк контекст\n",
    "from pyspark.sql import SparkSession\n",
    "\n",
    "spark = SparkSession.builder\\\n",
    "        .master('local[4]')\\\n",
    "        .appName('Lesson_2')\\\n",
    "        .config('spark.ui.port', '4050')\\\n",
    "        .config('spark.executor.instances', 2)\\\n",
    "        .config('spark.executor.memory', '5g')\\\n",
    "        .config('spark.executor.cores', 2)\\\n",
    "        .getOrCreate()\n",
    "\n",
    "sc = spark.sparkContext"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "colab": {
     "base_uri": "https://localhost:8080/",
     "height": 221
    },
    "id": "ZfhfV3EWHzqB",
    "outputId": "64488c68-78cc-429f-8aff-b0b216ed746d"
   },
   "outputs": [
    {
     "data": {
      "text/html": [
       "\n",
       "            <div>\n",
       "                <p><b>SparkSession - in-memory</b></p>\n",
       "                \n",
       "        <div>\n",
       "            <p><b>SparkContext</b></p>\n",
       "\n",
       "            <p><a href=\"http://c2e7eff26cd0:4050\">Spark UI</a></p>\n",
       "\n",
       "            <dl>\n",
       "              <dt>Version</dt>\n",
       "                <dd><code>v3.3.1</code></dd>\n",
       "              <dt>Master</dt>\n",
       "                <dd><code>local[4]</code></dd>\n",
       "              <dt>AppName</dt>\n",
       "                <dd><code>Lesson_2</code></dd>\n",
       "            </dl>\n",
       "        </div>\n",
       "        \n",
       "            </div>\n",
       "        "
      ],
      "text/plain": [
       "<pyspark.sql.session.SparkSession at 0x7f14123cdf40>"
      ]
     },
     "execution_count": 3,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "spark"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {
    "id": "VrBE3PdXtOwT"
   },
   "source": [
    "# Самостоятельная работа\n",
    "\n",
    "Требуется выяснить:\n",
    "1. Какое соотношение сторон экрана телефона самое популярное,\n",
    "2. Плотность пикселей у экрана.\n",
    "\n",
    "Можно использовать только rdd.\n"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {
    "id": "FjAfrNtXSKBz"
   },
   "source": [
    "## Считывание данных\n",
    "Данные взяты отсюда: https://www.kaggle.com/iabhishekofficial/mobile-price-classification\n",
    "Скачиваем и копируем в папку с нотебуком\n",
    "\n",
    "Внутри содержится следующая информация:\n",
    "\n",
    "* id: ID\n",
    "* battery_power: Total energy a battery can store in one time (mAh)\n",
    "* blue: Support bluetooth or not\n",
    "* clock_speed: Speed at which microprocessor executes instructions\n",
    "* dual_sim: Support dual sim or not\n",
    "* fc: Front Camera mega pixels\n",
    "* four_g: Support 4G or not\n",
    "* int_memory: Internal Memory (GB)\n",
    "* m_dep: Mobile Depth (cm)\n",
    "* mobile_wt: Weight of mobile phone\n",
    "* n_cores: Number of cores of processor\n",
    "* pc: Primary Camera mega pixels\n",
    "* px_height: Pixel Resolution Height\n",
    "* px_width: Pixel Resolution Width\n",
    "* ram: Random Access Memory (MB)\n",
    "* sc_h: Screen Height of mobile (cm)\n",
    "* sc_w: Screen Width of mobile (cm)\n",
    "* talk_time: Time that a single battery charge will last\n",
    "* three_g: Support 3G or not\n",
    "* touch_screen: Has touch screen or not\n",
    "* wifi: Support wifi or not"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "id": "3VyKjYOSkLlp"
   },
   "outputs": [],
   "source": [
    "# Считываем данные\n",
    "train = sc.textFile('/train.csv')\n",
    "test = sc.textFile('/test.csv')"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "id": "E9Vb0d0hutEb"
   },
   "outputs": [],
   "source": [
    "# Выведите содержимое каждого файла (часть строк)\n",
    "train.take(2)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "colab": {
     "base_uri": "https://localhost:8080/"
    },
    "id": "6D6W1RYbIJLF",
    "outputId": "50f61003-2573-4e6b-f939-b24dec0a3383"
   },
   "outputs": [
    {
     "data": {
      "text/plain": [
       "['id,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,pc,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi',\n",
       " '1,1043,1,1.8,1,14,0,5,0.1,193,3,16,226,1412,3476,12,7,2,0,1,0']"
      ]
     },
     "execution_count": 7,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "test.take(2)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {
    "id": "5pNi9czovHNh"
   },
   "source": [
    "##  Преобразуем train и test"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {
    "id": "P_DVQDecnNZP"
   },
   "source": [
    "Все функции расписаны тут: [lhttps://spark.apache.org/docs/latest/rdd-programming-guide.html](https://spark.apache.org/docs/latest/rdd-programming-guide.html). Так же хорошая статья [тут](https://sparkbyexamples.com/pyspark-rdd)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {
    "id": "VZNtczWznGh0"
   },
   "source": [
    "![image.png](...)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "id": "8PzsLhWdvCY_"
   },
   "outputs": [],
   "source": [
    "train_first_row = train.first()\n",
    "\n",
    "train = train\\\n",
    "    .filter(lambda row: row != train_first_row)\\\n",
    "    .map(lambda row: [float(el) for el in row.split(',')])"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "id": "QKCMFiVenNia"
   },
   "outputs": [],
   "source": [
    "# Преобразуйте test\n",
    "test_first_row = test.first()\n",
    "\n",
    "test = test\\\n",
    "    .filter(lambda row: row != test_first_row)\\\n",
    "    .map(lambda row: [float(el) for el in row.split(',')])"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {
    "id": "kjCdeAS4v1Vc"
   },
   "source": [
    "## Объединим train и test\n",
    "Найти нужную функцию можно [здесь](https://spark.apache.org/docs/3.1.1/api/python/reference/pyspark.html#rdd-apis)\n",
    "\n",
    "PS: нужно сделать средствами rdd pd.concat([train, test,], axis=0). Проверьте, что union вышел правильно"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "id": "5-Oeq54KxhXa"
   },
   "outputs": [],
   "source": [
    "# Объедините train и test\n",
    "data = train.union(test)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "colab": {
     "base_uri": "https://localhost:8080/"
    },
    "id": "fWhGgfSVI11r",
    "outputId": "c2f5389a-afb3-44be-b166-49a34cc513a7"
   },
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "общее кол-во строк 3000\n"
     ]
    }
   ],
   "source": [
    "print('общее кол-во строк', data.count())"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "id": "A_AwX16zcstx"
   },
   "outputs": [],
   "source": [
    "import pandas as pd"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "colab": {
     "base_uri": "https://localhost:8080/",
     "height": 291
    },
    "id": "W86Ebe-2cyBn",
    "outputId": "21175413-c248-4c17-ce0c-4ffe5ddab1cf"
   },
   "outputs": [
    {
     "data": {
      "text/plain": [
       "<matplotlib.axes._subplots.AxesSubplot at 0x7f140a1faca0>"
      ]
     },
     "execution_count": 13,
     "metadata": {},
     "output_type": "execute_result"
    },
    {
     "data": {
      "image/png": "iVBORw0KGgoAAAANSUhEUgAAAXcAAAEACAYAAABI5zaHAAAABHNCSVQICAgIfAhkiAAAAAlwSFlzAAALEgAACxIB0t1+/AAAADh0RVh0U29mdHdhcmUAbWF0cGxvdGxpYiB2ZXJzaW9uMy4yLjIsIGh0dHA6Ly9tYXRwbG90bGliLm9yZy+WH4yJAAAYS0lEQVR4nO3de7RcZX3G8e8jAeUiSSCnEZLoQUWQVRXiEUJxtWiqcquhgFSsEjGaLhfiBa3G21LXsjZaWwSltBHQ0CqKeCEFFDGAFmmAEwg3A3JAIEkhOXKJIt7QX//Y7ynDsPc5s+fMTCZvns9as86ed797v++emfeZPXv2nqOIwMzM8vK0Ld0BMzPrPIe7mVmGHO5mZhlyuJuZZcjhbmaWIYe7mVmGWgp3SdMkXSjpdklrJR0saTdJl0u6M/2dnupK0hmSRiTdLGludzfBzMyaqZXz3CUtB/47Is6WtAOwE/Ah4KGIWCppCTA9Ij4g6QjgFOAI4CDg9Ig4aLz1z5gxIwYHBye5KWZm25bVq1f/PCIGyuZNGO6SpgJrgOdGQ2VJdwCHRsT9kvYAroqIfST9e5o+v7leVRtDQ0MxPDxce8PMzLZlklZHxFDZvFYOy+wFjAJfknSjpLMl7QzMbAjsB4CZaXoWsK5h+fWprLlTiyUNSxoeHR1tdVvMzKwFrYT7FGAucFZEHAD8CljSWCHt0df6HYOIWBYRQxExNDBQ+qnCzMza1Eq4rwfWR8S16f6FFGG/MR2OIf3dlOZvAOY0LD87lZmZWY9MGO4R8QCwTtI+qWg+8BNgBbAwlS0ELkrTK4AT01kz84DN4x1vNzOzzpvSYr1TgK+kM2XuBk6ieGO4QNIi4F7g+FT3UoozZUaAx1JdMzProZbCPSLWAGXfyM4vqRvAyZPsl5mZTYKvUDUzy5DD3cwsQw53M7MMtfqFqvWhwSWXlJbfs/TIHvfEzPqN99zNzDLkcDczy5DD3cwsQw53M7MMOdzNzDLkcDczy5DD3cwsQw53M7MMOdzNzDLkcDczy5DD3cwsQw53M7MMOdzNzDLkcDczy5DD3cwsQw53M7MMOdzNzDLkcDczy5DD3cwsQw53M7MMOdzNzDLkcDczy5DD3cwsQy2Fu6R7JN0iaY2k4VS2m6TLJd2Z/k5P5ZJ0hqQRSTdLmtvNDTAzs6eqs+f+iojYPyKG0v0lwMqI2BtYme4DHA7snW6LgbM61VkzM2vNZA7LLACWp+nlwNEN5edFYRUwTdIek2jHzMxqajXcA/i+pNWSFqeymRFxf5p+AJiZpmcB6xqWXZ/KnkTSYknDkoZHR0fb6LqZmVWZ0mK9l0fEBkl/Alwu6fbGmRERkqJOwxGxDFgGMDQ0VGtZMzMbX0t77hGxIf3dBHwbOBDYOHa4Jf3dlKpvAOY0LD47lZmZWY9MGO6Sdpb0zLFp4NXArcAKYGGqthC4KE2vAE5MZ83MAzY3HL4xM7MeaOWwzEzg25LG6n81Ir4n6XrgAkmLgHuB41P9S4EjgBHgMeCkjvfazMzGNWG4R8TdwEtKyh8E5peUB3ByR3pnZmZtafUL1Z4ZXHJJafk9S4/scU/MzLZe/vkBM7MMOdzNzDLkcDczy5DD3cwsQw53M7MMOdzNzDLkcDczy5DD3cwsQw53M7MMOdzNzDLkcDczy5DD3cwsQw53M7MMOdzNzDLkcDczy5DD3cwsQw53M7MMOdzNzDLkcDczy5DD3cwsQw53M7MMOdzNzDLkcDczy5DD3cwsQw53M7MMTdnSHeiEwSWXlJbfs/TIHvfEzKw/tLznLmk7STdKujjd30vStZJGJH1d0g6p/Onp/kiaP9idrpuZWZU6h2XeBaxtuP9p4LSIeD7wMLAolS8CHk7lp6V6ZmbWQy2Fu6TZwJHA2em+gFcCF6Yqy4Gj0/SCdJ80f36qb2ZmPdLqnvvngPcDf0z3dwceiYjH0/31wKw0PQtYB5Dmb071n0TSYknDkoZHR0fb7L6ZmZWZMNwlHQVsiojVnWw4IpZFxFBEDA0MDHRy1WZm27xWzpY5BHitpCOAZwC7AqcD0yRNSXvns4ENqf4GYA6wXtIUYCrwYMd7bmZmlSbcc4+ID0bE7IgYBF4PXBERfwtcCRyXqi0ELkrTK9J90vwrIiI62mszMxvXZC5i+gBwqqQRimPq56Tyc4DdU/mpwJLJddHMzOqqdRFTRFwFXJWm7wYOLKnzG+B1HeibmZm1yT8/YGaWIYe7mVmGHO5mZhlyuJuZZcjhbmaWIYe7mVmGHO5mZhlyuJuZZcjhbmaWIYe7mVmGHO5mZhlyuJuZZcjhbmaWIYe7mVmGHO5mZhlyuJuZZcjhbmaWIYe7mVmGHO5mZhlyuJuZZcjhbmaWIYe7mVmGHO5mZhlyuJuZZcjhbmaWIYe7mVmGHO5mZhmaMNwlPUPSdZJuknSbpE+k8r0kXStpRNLXJe2Qyp+e7o+k+YPd3QQzM2vWyp77b4FXRsRLgP2BwyTNAz4NnBYRzwceBhal+ouAh1P5aamemZn10IThHoVH093t0y2AVwIXpvLlwNFpekG6T5o/X5I61mMzM5tQS8fcJW0naQ2wCbgcuAt4JCIeT1XWA7PS9CxgHUCavxnYvWSdiyUNSxoeHR2d3FaYmdmTtBTuEfGHiNgfmA0cCOw72YYjYllEDEXE0MDAwGRXZ2ZmDWqdLRMRjwBXAgcD0yRNSbNmAxvS9AZgDkCaPxV4sCO9NTOzlrRytsyApGlpekfgVcBaipA/LlVbCFyUplek+6T5V0REdLLTZmY2vikTV2EPYLmk7SjeDC6IiIsl/QT4mqRPAjcC56T65wD/IWkEeAh4fRf6bWZm45gw3CPiZuCAkvK7KY6/N5f/BnhdR3pnZmZt8RWqZmYZcribmWXI4W5mliGHu5lZhhzuZmYZcribmWXI4W5mliGHu5lZhhzuZmYZcribmWXI4W5mliGHu5lZhhzuZmYZcribmWXI4W5mliGHu5lZhhzuZmYZcribmWXI4W5mliGHu5lZhhzuZmYZcribmWXI4W5mliGHu5lZhhzuZmYZcribmWXI4W5mlqEJw13SHElXSvqJpNskvSuV7ybpckl3pr/TU7kknSFpRNLNkuZ2eyPMzOzJWtlzfxx4b0TsB8wDTpa0H7AEWBkRewMr032Aw4G9020xcFbHe21mZuOaMNwj4v6IuCFN/xJYC8wCFgDLU7XlwNFpegFwXhRWAdMk7dHxnpuZWaVax9wlDQIHANcCMyPi/jTrAWBmmp4FrGtYbH0qa17XYknDkoZHR0drdtvMzMbTcrhL2gX4JvDuiPhF47yICCDqNBwRyyJiKCKGBgYG6ixqZmYTaCncJW1PEexfiYhvpeKNY4db0t9NqXwDMKdh8dmpzMzMeqSVs2UEnAOsjYh/aZi1AliYphcCFzWUn5jOmpkHbG44fGNmZj0wpYU6hwBvAm6RtCaVfQhYClwgaRFwL3B8mncpcAQwAjwGnNTRHpuZ2YQmDPeIuBpQxez5JfUDOHmS/TIzs0nwFapmZhlyuJuZZcjhbmaWIYe7mVmGHO5mZhlyuJuZZcjhbmaWIYe7mVmGHO5mZhlyuJuZZcjhbmaWIYe7mVmGHO5mZhlyuJuZZcjhbmaWIYe7mVmGHO5mZhlyuJuZZcjhbmaWIYe7mVmGHO5mZhlyuJuZZcjhbmaWIYe7mVmGHO5mZhlyuJuZZcjhbmaWoQnDXdK5kjZJurWhbDdJl0u6M/2dnsol6QxJI5JuljS3m503M7Nyrey5fxk4rKlsCbAyIvYGVqb7AIcDe6fbYuCsznTTzMzqmDDcI+JHwENNxQuA5Wl6OXB0Q/l5UVgFTJO0R6c6a2ZmrWn3mPvMiLg/TT8AzEzTs4B1DfXWp7KnkLRY0rCk4dHR0Ta7YWZmZSb9hWpEBBBtLLcsIoYiYmhgYGCy3TAzswbthvvGscMt6e+mVL4BmNNQb3YqMzOzHmo33FcAC9P0QuCihvIT01kz84DNDYdvzMysR6ZMVEHS+cChwAxJ64GPAUuBCyQtAu4Fjk/VLwWOAEaAx4CTutBnMzObwIThHhEnVMyaX1I3gJMn2ykzs342uOSS0vJ7lh7Z455U8xWqZmYZcribmWVowsMyZmZVtobDE9uqbTLc/YLctmyrz3cu253LdvSaD8uYmWVom9xzt+6pu5flvTKz7nC421YnhzeEdrYhh+223nG4t6BqUIEHlpn1J4e7WZN23sy9V23j2RKvD4d7l/jYs21tevEJ1a/z3nG4WyUfjrKtUad2rMZbZmvgcN+GeK/JbNvh89zNzDLkcDczy5DD3cwsQw53M7MMOdzNzDLkcDczy5DD3cwsQw53M7MM+SImM7M+NNmLDr3nbmaWIYe7mVmGHO5mZhlyuJuZZcjhbmaWIYe7mVmGuhLukg6TdIekEUlLutGGmZlV63i4S9oOOBM4HNgPOEHSfp1ux8zMqnVjz/1AYCQi7o6I3wFfAxZ0oR0zM6ugiOjsCqXjgMMi4q3p/puAgyLiHU31FgOL0919gDtKVjcD+HnNLtRdptv1c2mjH/vUizb6sU+9aKMf+9SLNvqxT+Mt85yIGChdIiI6egOOA85uuP8m4Attrmu428t0u34ubfRjn7zd/VM/lzb6sU/tLtONwzIbgDkN92enMjMz65FuhPv1wN6S9pK0A/B6YEUX2jEzswod/1XIiHhc0juAy4DtgHMj4rY2V7esB8t0u34ubfRjn3rRRj/2qRdt9GOfetFGP/aprWU6/oWqmZlteb5C1cwsQw53M7MMOdzNzDLUl+EuaTdJu/WgnV0lvVTS9C62MaNG3emSdu1WX2zrJWmmpLnpNnNL96dM3TEr6bXd6ks3SZrSML2LpKFu5NVkc7Bvwl3SsyV9TdIocC1wnaRNqWywpP5bGqZnS1op6RFJ10h6QUUb/zkWtpJeA9wKfBpYI+l1JfUfknS2pPmS1MI2HC7pZ5KulnSApNuAayWtlzS/Ypk9JZ0naTPFFWi3SrpP0sclbT9Be3094D3YO7Lu/SWtAq4CPpNuP5S0StLcNta3S4f6dYiktZJuk3SQpMuB6yWtk3RwSf1jmm7HAsvG7rfYZt3X0/MlHVv121aSptVZX1rmzcBGST+VdDhwM0WG3CTphHGWG0iZ8OLxnoO6OTiuulc9desG/A/wN8B2DWXbUZwnv6qk/g0N0xdQ/JTB04C/BlZWtHFLw/Q1wGCangHcVFL/DuAdwI8pLsQ6HZg3zjasAV4IHAw8OFY3ld1QscwVwKFp+hjgNGBn4JPAsopl9gdWAWuBH6Tb7alsbs3HfZcOPX+HpP7cBhwEXA7cBawDDi6pf0zT7VjggbH7NdrdrUbd56d29hunzrSa2/3m9Fz/lOLH8u4GVqbtPmGc5QaAA4AXj/ccpNfUQSXl88pesy30976Sshel1846ilPupjfMu65iPdel5Q6m2Cl5eSqfC/y4pP7vgYuBc4Evpdsv099zS+p/pGF6v/T4/gy4p+zxSPWuBGak6TelZc4GbgFOKan/eBo7i1p93tO6ZgB7Ab8AnpfKZwI3l9TfL7UxAvyOIrB/BnwZmFpSv1YOjtvXui+Obt2AO+vM48nhvqZp3o0V67kN2DVNXw08rXHeBG08G3g/cEMawJ+aoP66pnlrKvp0U9P91Q3Tt1cs07EBXzbYU3mtAd/twZ6WqTXg6w72VK/WgO/BYB9vXIxUlJ9acXsv8FBJ/auBw4BpwPvSOBnbjqqxdGPD9NqqcdBQ9jKKN723N5T9bJxtaxxLlwCHp+kDgWsqlrm1Yfp6YPc0vVPFc3ELcBTwFYo36IsoQnTHcfq1pmH6f5vmlbWxCtinoe/L0/TbgAtrPt+V88puHb+IaRJWS/pXYDlFoEDxMwYLgRtL6s+WdAYgYEDS9hHx+zSv6nDGJ4ArJZ1JsTf+DUkrgFcA3yup//+HYiLiPtLHYkn7Ury7NntE0t8BuwIPS3oPxaeKvwQerejTqKQ3UgTRMRRBRToMVHXYbOeIuLa5MCJWSdr5KRshnVqxHgFVHxHPAj5O8eJ8K3C1pNdGxF2UP77bR8Qtqb3RiLg69ekGSTuW1P8zYClwfUSclZY7NCJOqugPFI/PJ9P0PwHviojvSjoQ+FxaZ6OBiBj7saV3UnyCeFDSTmm7Pl/Sxtq0rhMonuurgfOBiyLi1yX1/5Da+LmkR9PjQ0RsrDiSdy6wMCLuSP0+OSIOkvQ24ByK32Zq9F1JlwDn8eRxcSLlr1mAT1E8Po+XzCt7TT0zIsbW9VlJq4HvqfjRv6oLYRrX88GmeTs0V46I6yW9CjhF0pXAB8ZZd7M9I+K7aT3XVbyeAH4vaVZEbKAYb79K5b+l2Pt9Sv2IuBi4OK3zryjC/UxJl0XEG0qWuU/SPwLPBG6X9M/AtyjG+P0l9XeMiDsa+v5vafqLFeOybg5W6qdwP5Fib+kTwKxUth74L4oXfbO/b5gepgiphyU9i4qfO4iICyTdQPGu+QKK7Z8HnB8Rl5UscmXFem5P/Wy2EPgI8Efg1RQBcRlwb2qzzFuAzwJLKPbIx349czeeOmjG1B3wdQc71B/wvRzs0NqArzvYof6A7+pgj4h3pmO7C3hiXGwAzoyISyu24QbgOxGxunmGpLeWLSBpakRsTm1emY6Jf5PidVjmo5J2iojHIuI7Det5HsXr8iki4o/A6ZK+QfEGOp7nph0vUezI7RQRj6V5VTtv7wG+L+mbFJ8+rpB0GfByik+EzRp33n5NsSN2gaSpwNEVbbwROBnYTDFmX0Pxer+X4hBds7skfZTi8OsxFGOc9H1a2dirm4OVfIXqVqpiwK8oG/CSrqE4DFE22NdFxJyS8puAPx8b8KnsxaQBHxG7N9V/LfCDhgE4Vv484NiI+Mw427InxWAfiojnjlPvEeBHFINyHsXPnT6W5t0aEX/aVP9Qin8cMxZScynebF8OXBYRny1p48aIOKCkfCpwdEQsbyrflWKwB/AFisF+EsVg/2RE3N9U/1sUe2Bjg316RLwlDfZbI2Kfqu1vlaR9gAcbPrU0zpsZERubyt4A3B0Rq5rKnw18NCKqdky6RtJfNBWtjohHVZw4cFxEnFmx3FTgDTyx87ae4lPX7SV131f2Guik9KXthygOx90ELI2IX6Z+vrD5Me+oOsdwttQNOKqb9XvRRi/6NM569iEdey6ZN7Oi/A2UfHlM8d3DF7fQ6+Avmm67jG0DxeGNsmWmAm+n+KL68xSfEPYdp433dXkbplEc3rsY+AeKT0hj/az8sr5iXYu3xPPQ6X7163Z0e7vbWH+tPOibUyEn8LIu1+9FG73o09g/QXmSiLgjSvbi0ryNFeVfjZK9ioi4L2ruyZX1qZ36EfHDptujqXxjVOzJRcTmiDgrIt4TEadExKejZC+uoX7H9uQqnotHIuL9EXFURHw4In7Z0M+6e3ETnp7bSp86WX9ssW7Wb6dP/bjdbbRRKw/66Zg76YvKskMNH+tE/V600Ys+TaD2Cywiav3iXBvLdHWwQ/0+9eN2V60/vT5mAdeOvaEl99ZZfzt9Gq9+3X51cDtqvz7aWKZj2123jfRFe0TxvdR+FGcy3V43D/pmz13SByj+36ooTqu7Lk2fL2nJZOv3oo1e9KkFv6tZvxcDpW6f6taHHryBtLHMpJ8LSe+kOEXvFIoL3Br/H/GnKlck7avi4rvms6Eqg7dm/Vr9amc76vapze3o6nbXbUPSx4AzgLPSF/VfoLjuZYmkD5etv9KWPk7VcDzppxSn0zWX70D5ee616veijV70qYXHsfS89XHqn9RGG7WWaaNPteq32ad+3O6nrJ/iXOyx7xYGKc4Me1e6X3UO+jspLsD7DsWptQsa5pWdg16rfjv9aqN+O33qx+2u26dbKM7k2oniuomx63J2pOQ8+vFu/XRY5o/Anjz13WyPNG+y9XvRRi/6hKSbK9Ylii8X6/gE5aeJ1Vqmbp86vA2lfepw/dJlevBcPC2e+G7hnnQG0IWSnkP1J4m3AS+N4uySwVR/MCJOr1imbv12+lW3fjt96sftrtvG4xHxB+AxSXdFxC9SW7+WVJUhpfop3N8NrJR0J0+cu/1sikvG39GB+r1ooxd9giI0XgM83FQuip9VeHJhGwHUxjK1+tRG/Z68gXR7u9tY/0ZJ+0fEGoAUEkdRXAz1oop1dTt42+lX3frt9Kkft7tuG7/TE+f0v3SsUMWpk7XCvdbH0m7fKL4DmEfx+x/HpuntOlW/F230qE/nkC7vL5n31ZKyjRS/R/OcptsgTZdQt7tMG32qVb/NPvXjdtdd/2zgWRXrP6Si/Apg/6ayKRQXF/1hsvXb6Vcb9dvpUz9ud90+Pb1i3TOAF5XNq7q1XNG3rffWZpDWXqbftqMft7sXj2u3g7dHz3U7b2p9t91b8rH1FapmZhnqm1MhzcyscxzuZmYZcribmWXI4W5mlqH/A49z/n2fPE74AAAAAElFTkSuQmCC\n",
      "text/plain": [
       "<Figure size 432x288 with 1 Axes>"
      ]
     },
     "metadata": {
      "needs_background": "light"
     },
     "output_type": "display_data"
    }
   ],
   "source": [
    "pd.Series(data.map(lambda row: (row[2], 1)).countByKey()).sort_index().plot.bar()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "colab": {
     "base_uri": "https://localhost:8080/"
    },
    "id": "fJe7zskxeR0p",
    "outputId": "3fb51e68-7403-4755-eb80-d989b9715a3f"
   },
   "outputs": [
    {
     "data": {
      "text/plain": [
       "['battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,pc,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range']"
      ]
     },
     "execution_count": 14,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "train_first_row.split()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "colab": {
     "base_uri": "https://localhost:8080/"
    },
    "id": "hOpMKRsRdkyV",
    "outputId": "08077587-af67-4091-be1e-02ead117013d"
   },
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "кол-во колонок в трейн 21 кол-во колонок в тест 21\n",
      "общ.колонки {'pc', 'clock_speed', 'four_g', 'ram', 'blue', 'int_memory', 'sc_w', 'fc', 'mobile_wt', 'dual_sim', 'battery_power', 'px_width', 'id', 'px_height', 'sc_h', 'n_cores', 'three_g', 'touch_screen', 'wifi', 'm_dep', 'talk_time', 'price_range'}\n",
      "отл. в заголовках трейн от тест {'price_range'}\n",
      "отл. в заголовках тест от трейн {'id'}\n",
      "отл. в заголовках ['price_range'] ['id']\n"
     ]
    }
   ],
   "source": [
    "train_header = train_first_row.split(',')\n",
    "test_header = test_first_row.split(',')\n",
    "print('кол-во колонок в трейн', len(train_header), 'кол-во колонок в тест', len(test_header))\n",
    "print('общ.колонки', set(train_header) | set(test_header))\n",
    "print('отл. в заголовках трейн от тест', set(train_header) - set(test_header))\n",
    "print('отл. в заголовках тест от трейн', set(test_header) - set(train_header))\n",
    "\n",
    "diff = lambda l1,l2: filter(lambda x: x not in l2, l1)\n",
    "print('отл. в заголовках', list(diff(train_header, test_header)), list(diff(test_header, train_header)))"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "id": "X40pFtCJsupU"
   },
   "outputs": [],
   "source": [
    "train = train.map(lambda row: row[:-1])\n",
    "test = test.map(lambda row: row[1:])\n",
    "data = train.union(test)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "id": "WTSTXyXbtWOl"
   },
   "outputs": [],
   "source": [
    "train_header = train_header[:-1]"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "colab": {
     "base_uri": "https://localhost:8080/",
     "height": 289
    },
    "id": "xPnQy-S6thKb",
    "outputId": "fc98758a-5dd0-4807-9c94-17227558036a"
   },
   "outputs": [
    {
     "data": {
      "text/plain": [
       "<matplotlib.axes._subplots.AxesSubplot at 0x7f140a05e8b0>"
      ]
     },
     "execution_count": 18,
     "metadata": {},
     "output_type": "execute_result"
    },
    {
     "data": {
      "image/png": "iVBORw0KGgoAAAANSUhEUgAAAXcAAAD+CAYAAADBCEVaAAAABHNCSVQICAgIfAhkiAAAAAlwSFlzAAALEgAACxIB0t1+/AAAADh0RVh0U29mdHdhcmUAbWF0cGxvdGxpYiB2ZXJzaW9uMy4yLjIsIGh0dHA6Ly9tYXRwbG90bGliLm9yZy+WH4yJAAAWfklEQVR4nO3df7RdZX3n8feXBCqCJEDupJAfhgqCrLYCRoTqaqmZtgapoYJUnUrKpJNZXeDYUitpV13UGduiU0tlZOiiYBtaW6SoJQLFYkjXLOoESCCAGJCIQJICuSJQKfUH+p0/9nPHw83Z955777knN0/er7X2uns/+/me/Zxz9/mcfffZ59zITCRJddlvTw9AktR/hrskVchwl6QKGe6SVCHDXZIqNHtPDwBg3rx5uWTJkj09DEnaq2zevPkbmTnUbd2MCPclS5awadOmPT0MSdqrRMRjbes8LSNJFTLcJalChrskVchwl6QKGe6SVCHDXZIqZLhLUoUMd0mqkOEuSRWaEZ9Q7bRkzU2t6x695K0DHIkk7b16OnKPiLkRcX1EPBgRWyPi1Ig4LCJujYiHy89DS9+IiMsiYltE3BcRJ03vXZAkjdbraZmPA7dk5nHAa4GtwBpgfWYeA6wvywDLgWPKtBq4oq8jliSNa9xwj4g5wE8DVwNk5ncz81lgBbC2dFsLnFnmVwDXZGMjMDcijuj7yCVJrXo5cj8KGAb+IiLuiYirIuIgYH5mPlH6PAnML/MLgO0d9TtK20tExOqI2BQRm4aHhyd/DyRJu+kl3GcDJwFXZOaJwL/xw1MwAGRmAjmRDWfmlZm5NDOXDg11/TpiSdIk9RLuO4AdmXlHWb6eJuyfGjndUn7uKut3Aos66heWNknSgIwb7pn5JLA9Io4tTcuArwDrgJWlbSVwQ5lfB5xbrpo5BXiu4/SNJGkAer3O/b3ApyLiAOAR4DyaF4brImIV8BhwTul7M3A6sA14ofSVJA1QT+GemVuApV1WLevSN4HzpzguSdIU+PUDklQhw12SKmS4S1KFDHdJqpDhLkkVMtwlqUKGuyRVyHCXpAoZ7pJUIcNdkipkuEtShQx3SaqQ4S5JFTLcJalChrskVchwl6QKGe6SVCHDXZIqZLhLUoUMd0mqkOEuSRUy3CWpQoa7JFXIcJekCvUU7hHxaETcHxFbImJTaTssIm6NiIfLz0NLe0TEZRGxLSLui4iTpvMOSJJ2N5Ej95/NzBMyc2lZXgOsz8xjgPVlGWA5cEyZVgNX9GuwkqTeTOW0zApgbZlfC5zZ0X5NNjYCcyPiiClsR5I0Qb2GewL/GBGbI2J1aZufmU+U+SeB+WV+AbC9o3ZHaZMkDcjsHvu9KTN3RsR/AG6NiAc7V2ZmRkROZMPlRWI1wOLFiydSKkkaR09H7pm5s/zcBXwOOBl4auR0S/m5q3TfCSzqKF9Y2kbf5pWZuTQzlw4NDU3+HkiSdjNuuEfEQRHxipF54OeBLwPrgJWl20rghjK/Dji3XDVzCvBcx+kbSdIA9HJaZj7wuYgY6f83mXlLRNwFXBcRq4DHgHNK/5uB04FtwAvAeX0ftSRpTOOGe2Y+Ary2S/vTwLIu7Qmc35fRSZImxU+oSlKFDHdJqpDhLkkVMtwlqUKGuyRVyHCXpAoZ7pJUIcNdkipkuEtShQx3SaqQ4S5JFTLcJalChrskVchwl6QKGe6SVCHDXZIqZLhLUoUMd0mqkOEuSRUy3CWpQoa7JFXIcJekChnuklQhw12SKmS4S1KFeg73iJgVEfdExI1l+aiIuCMitkXEpyPigNL+I2V5W1m/ZHqGLklqM5Ej9/cBWzuWPwJcmplHA88Aq0r7KuCZ0n5p6SdJGqCewj0iFgJvBa4qywG8Gbi+dFkLnFnmV5Rlyvplpb8kaUB6PXL/U+ADwA/K8uHAs5n5YlneASwo8wuA7QBl/XOl/0tExOqI2BQRm4aHhyc5fElSN+OGe0ScAezKzM393HBmXpmZSzNz6dDQUD9vWpL2ebN76PNG4G0RcTrwMuAQ4OPA3IiYXY7OFwI7S/+dwCJgR0TMBuYAT/d95JKkVuMeuWfm72TmwsxcArwTuC0z/xOwATi7dFsJ3FDm15VlyvrbMjP7OmpJ0pimcp37RcCFEbGN5pz61aX9auDw0n4hsGZqQ5QkTVQvp2X+v8z8J+CfyvwjwMld+nwbeEcfxiZJmiQ/oSpJFTLcJalChrskVchwl6QKGe6SVCHDXZIqZLhLUoUMd0mqkOEuSRUy3CWpQoa7JFXIcJekChnuklQhw12SKmS4S1KFDHdJqpDhLkkVMtwlqUKGuyRVyHCXpAoZ7pJUIcNdkipkuEtShQx3SaqQ4S5JFRo33CPiZRFxZ0TcGxEPRMSHSvtREXFHRGyLiE9HxAGl/UfK8rayfsn03gVJ0mi9HLl/B3hzZr4WOAF4S0ScAnwEuDQzjwaeAVaV/quAZ0r7paWfJGmAxg33bDxfFvcvUwJvBq4v7WuBM8v8irJMWb8sIqJvI5Ykjaunc+4RMSsitgC7gFuBrwHPZuaLpcsOYEGZXwBsByjrnwMO73KbqyNiU0RsGh4entq9kCS9RE/hnpnfz8wTgIXAycBxU91wZl6ZmUszc+nQ0NBUb06S1GFCV8tk5rPABuBUYG5EzC6rFgI7y/xOYBFAWT8HeLovo5Uk9aSXq2WGImJumT8Q+DlgK03In126rQRuKPPryjJl/W2Zmf0ctCRpbLPH78IRwNqImEXzYnBdZt4YEV8Bro2IDwP3AFeX/lcDfxUR24BvAu+chnFLksYwbrhn5n3AiV3aH6E5/z66/dvAO/oyOknSpPgJVUmqkOEuSRUy3CWpQoa7JFXIcJekChnuklQhw12SKmS4S1KFDHdJqpDhLkkVMtwlqUKGuyRVyHCXpAoZ7pJUIcNdkipkuEtShQx3SaqQ4S5JFTLcJalChrskVchwl6QKGe6SVCHDXZIqZLhLUoUMd0mq0LjhHhGLImJDRHwlIh6IiPeV9sMi4taIeLj8PLS0R0RcFhHbIuK+iDhpuu+EJOmlejlyfxH4rcw8HjgFOD8ijgfWAOsz8xhgfVkGWA4cU6bVwBV9H7UkaUzjhntmPpGZd5f5bwFbgQXACmBt6bYWOLPMrwCuycZGYG5EHNH3kUuSWk3onHtELAFOBO4A5mfmE2XVk8D8Mr8A2N5RtqO0jb6t1RGxKSI2DQ8PT3DYkqSx9BzuEXEw8BngNzLzXzvXZWYCOZENZ+aVmbk0M5cODQ1NpFSSNI6ewj0i9qcJ9k9l5mdL81Mjp1vKz12lfSewqKN8YWmTJA1IL1fLBHA1sDUz/6Rj1TpgZZlfCdzQ0X5uuWrmFOC5jtM3kqQBmN1DnzcC7wHuj4gtpe13gUuA6yJiFfAYcE5ZdzNwOrANeAE4r68jliSNa9xwz8zbgWhZvaxL/wTOn+K4JElT4CdUJalChrskVchwl6QKGe6SVCHDXZIqZLhLUoUMd0mqkOEuSRUy3CWpQoa7JFXIcJekChnuklQhw12SKmS4S1KFDHdJqpDhLkkVMtwlqUKGuyRVyHCXpAoZ7pJUIcNdkipkuEtShWbv6QFI0t5iyZqbWtc9eslbBziS8VUR7nvTAy5Jg1BFuEttL/D7you7Bzgabdxwj4hPAmcAuzLzx0vbYcCngSXAo8A5mflMRATwceB04AXgVzPz7ukZuvYGho4mY19/se6HXo7c/xL4BHBNR9saYH1mXhIRa8ryRcBy4JgyvQG4ovxUBQxquQ/sPca9WiYz/w/wzVHNK4C1ZX4tcGZH+zXZ2AjMjYgj+jVYSVJvJnvOfX5mPlHmnwTml/kFwPaOfjtK2xOMEhGrgdUAixcvnuQwJGlm21N/7Uz5OvfMTCAnUXdlZi7NzKVDQ0NTHYYkqcNkj9yfiogjMvOJctplV2nfCSzq6LewtEk9m8nndSfzRp9vDg6Gj/NLTTbc1wErgUvKzxs62i+IiGtp3kh9ruP0zYwyqB3BHW7mmqm/m5n84jYZM/Vxrl0vl0L+LXAaMC8idgAX04T6dRGxCngMOKd0v5nmMshtNJdCnjcNY95jfEFQTWp7EZmMmp9r44Z7Zr6rZdWyLn0TOH+qg9LE1byTSpo4P6G6j/KoTaqb3wopSRUy3CWpQoa7JFXIcJekChnuklQhw12SKmS4S1KFDHdJqpDhLkkVMtwlqUJ+/YAkzTD9+HoQj9wlqUKGuyRVyHCXpAoZ7pJUIcNdkipkuEtShQx3SaqQ4S5JFTLcJalChrskVchwl6QKGe6SVCHDXZIqNC3hHhFviYiHImJbRKyZjm1Iktr1PdwjYhZwObAcOB54V0Qc3+/tSJLaTceR+8nAtsx8JDO/C1wLrJiG7UiSWkRm9vcGI84G3pKZv1aW3wO8ITMvGNVvNbC6LB4LPNRyk/OAb0xgCBPtb83gambquKyZueOyZuz+r8zMoa5rMrOvE3A2cFXH8nuAT0zh9jZNZ39rBlczU8dlzcwdlzWT20ZmTstpmZ3Aoo7lhaVNkjQg0xHudwHHRMRREXEA8E5g3TRsR5LUou//IDszX4yIC4AvALOAT2bmA1O4ySunub81g6uZqeOyZuaOy5rJbaP/b6hKkvY8P6EqSRUy3CWpQoa7JFXIcJ+kiJi3p8cgqV4RcVhEHDbZ+qrCPSIOiYjXRcShfb7d5RHx9Yi4PSJOjIgHgDsiYkdELOuh/tCIOKSfY1K9ImJ+RJxUpvlTuJ2D+zmuLrc/4eCJiLdN93Yi4uiIOGus77SKiLmTGMfsjvmDI2JpL2OLiKGSGz853u8kIhZHxLURMQzcAdwZEbtK25IJDXgyn3yajgn4zx3zC4H1wLPAl4BXt9T8NTCvzP8C8DjwReAx4B0tNd8ErgKWUa4W6mFsW4DXAKcCTwOnlPbXAHe31BwJXAM8B3y/jO1x4PeB/cfZ3nzgpDLNn8JjevA0/84Om0TN26ZzG8DRwFnA8WP0mTvJ+zu787EFlvYyPmAIOBH4ybF+J8AJwEZga9mPvwg8WNpOmsR4H29p/4lym9tpLrM7tGPdnS01byzjegB4A3Ar8LVyG6e21Lx91HQW8OTIckvN73XMHw98Ffg68CjN15h0q9nQkQPvKTVXAfcD722pebE8vqt62R+AXy3P/a/SfCniIzQZtR14V0vN8WUb24Dv0oT114G/BOa01Pxf4JeBWR1ts2g+L7RxQr//yezk0zHREZLAdTTfO7Mf8EvA+paa+zvmvwQsKfPzgHtbah4CLgD+meaTsx+nhHWPY9s+at2WlprbgNM6dvJLgYOADwNXttTs00/umfrELjXT/uSmOYjY7X4Cp4yxP1/YMv0W8M2WmtuBtwBzgfeX3+mryrp7WmruLPvNqTTfc/Km0n4S8M8tNd8DbgQ+CfxFmb5Vfn6yh+faTcDyMn8y8KWWmi93zN8FHF7mXw7c11JzP3AG8Knye72BJkAPHKP/POAo4F87Hq/5Y2xjI3Bsx/jXlvn/AlzfUvPwGPtg67qu/SfSeTqnUb/ULaPWte1wDwCHdOyw+3Wu62E7i4EPAHeXJ+sfttTcBvxX4LeB+4DfBBYAK4HbW2ruHbW8uWP+wZaaffrJPVOf2B010/rkHueJva2l/dvA/wAu7jI92+O++bPAw2U/a/tL9J6O+a1tv7dR7a+neQH89Y62r7fdxy77wD1tYxjdDiwo8xuAl5X5WfSWAwcC5wCfLfvD33Tpv6Vj/l9GrWv7/Y9+nDu3ubWl5lrgf9McQB1ZpjeUtuvGeuxGT33/hOoULIyIy4AAhiJi/8z8Xlm3f0vNh4ANEXE5zZH430XEOpqd9ZaWmhiZyczHgY8CH42I42j+HOpmJfB7wA+AnwfeRfMJ3MdonqjdDEfEr9DsbG+nOfokIoL29zoOysw7Rjdm5saIOKil5g+B/0lzNDpa23ZekZkjj88fR8Rm4JbyDZ7ZUrN/Zt5f7sNwZt5exnZ3RBzYUvNTwCXAXZl5Rak9LTPPa+nf6cjM/IeyjTvH2Mb3ImJBZu4Engf+rbR/h+bJ3bUmM28Ebiy3+4s04X55RHwhM9/dpeb7mfkN4BsR8Xxmfq2M7anmV9rVgZn5UMd9+LMy/+cRcWGX/v8QETfRnM7bXtoWAefSvj/fDfx9Zm4evSIifq1tYBExJzOfK+PZEBFnAZ8B2s4hd+5LvzNq3QHdCjLzroj4OeC9EbEBuIj2/WvEj5XncNBkwssz84Wyri0HfhP4x4j4DM2Bym0R8QXgTTQHEt105sC/05wtuC4i5gBndun/eET8EfAK4MGI+BjNi8F/BJ5o2cbXIuKDNAeHb6c5eCMi9qf9uXkuzV+UH6I5gATYAXweuLqlpruJvBJM50QToJ3ToaX9R2k5oi7rjwY+AnyuPABXAL8wRv8/GdD9WUyzw3yZ5r2BI0r74cBZLTWX0Ryx/jJNMP5Umb+Jlm/WpDkd9bqWddtb2u9l1GkBmvPBDwNPt9V0zJ85at2Xu9WUdfsB76N5kTsZeGSMvs/SfA/R54Fh4OXjbQM4jeYJ/d+BT5TH42Ka00bvb6lpOwKcA6xsWbcO+KOyjduAj9GcqroY+EJLzWeBD5Z+H6P8tUITUg+11CwH/qw8Bp8v86eP8ZgdSzkt1WVd1/drgHfT5VRk2Wf/vKXmbZ2/j472VwEf6OH5cGR5PrT+/ku/nxk1HTxyX4Dzx6ibA/w6zenP/0XzQnLcGP277htj9D+E5kVtDc37LWfR/FV6OeW53aVmLs3B443AH9AcVI2MdcxTwf2Y/PqBKYqIM7I5CuzX7S2n+ecmI6/aO4F1mXlzS/9jaQJ5t+97joj5mflUl/Z30zzJNo5qXwx8MDN3+2ukXOXwxfzhUdRI+6toXqw+Os79OhL4U2BpZv5YS5+fGdW0OTOfL1eMnJ2Zl7fUzaEJrFfTfF/SDuCGzHywpf/7M/OPxxpvl5pDgPNpjjw/QfMG/nk0f719ODN3O3orV2T8Ls2593uBSzLzW2W8rxn9+EtjmXDWTPerRz8m4IwZXPOhQWzHad+egNXWzMyaAY5rQlmzt1zn/vo9XRMRx0XERRFxWZkuiojXZObFgxhb+c9V+2zNTB3XAGtaT+xbs8dr+rqNiDg5Il5f5o+PiAsj4vSJZs1MekOV8qZmt1MSrXdqEDURcRHNm6jX0lw1As21+H8bEddm5iX9GtsYZuqOPaiamTquvtaUfWYBcEdmPt+x6rHWG7JmIDUD2sbFNO+7zI6IW2mulNkArImIEzPzD9q2tZuJ/mkwXRPNGyBbaN6w+JUyrRlp28M1X6XLB49orhLoevnaZLYzzuNz3r5cM1PH1c8a4L/RfA7j72murlrRsa7tckNrBlAzwHHdT3OV18tpLrkdudT7QFouuWzdxya6U07XxOQCdFA1D9L8I9rR7a+k/aqHCW9nnMen6weS9pWamTquftaUJ/bI1SFLgE3A+8py2xU+1gygZoDjuqfbfFnu+oHJtmkmnZb5Ac3lUqP/XDmirNuTNb8BrI+Ih/nh9ceLaS7DvKBf24mI+1puK2guBau6ZqaOa4A1+2X50z0zH42I04DrI+KVtJ/6sWYwNYMa13c7ru1/3UhjucKqLZ+6mknhPpkAHUhNZt4SEa+muU678/z5XZn5/T6ObT7NJXbPjGoPmuu3a6+ZqeMaVM1TEXFCZm4ByOYy0DNoPuH7Ey3bsGYwNYMa109n5ndK/84w35/m8z+9m8hh/nRPNB94OYXmAwJnlflZM6FmEPeH5hNob2pZt9tHomurmanjGuD9Xwj8aEv/N7a0WzOAmkGNq5+TH2KSpArtLde5S5ImwHCXpAoZ7pJUIcNdkir0/wDjIsArEvI7bgAAAABJRU5ErkJggg==\n",
      "text/plain": [
       "<Figure size 432x288 with 1 Axes>"
      ]
     },
     "metadata": {
      "needs_background": "light"
     },
     "output_type": "display_data"
    }
   ],
   "source": [
    "pd.Series(data.map(lambda row: (row[2], 1)).countByKey()).sort_index().plot.bar()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {
    "id": "GxGGHqRyx7XG"
   },
   "source": [
    "## Рассчитайте соотношение сторон телефона и экрана"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "colab": {
     "base_uri": "https://localhost:8080/"
    },
    "id": "0cZsXlIruXtU",
    "outputId": "85856c09-14a3-40bd-96e2-06925153ff26"
   },
   "outputs": [
    {
     "data": {
      "text/plain": [
       "(14, 15)"
      ]
     },
     "execution_count": 19,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "sc_h = train_header.index('sc_h')\n",
    "sc_w = train_header.index('sc_w')\n",
    "sc_h, sc_w"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "colab": {
     "base_uri": "https://localhost:8080/"
    },
    "id": "t5WOnetOuvss",
    "outputId": "6f321822-ca70-4d45-abfb-233c38fe48ed"
   },
   "outputs": [
    {
     "data": {
      "text/plain": [
       "[1.2857142857142858,\n",
       " 5.666666666666667,\n",
       " 5.5,\n",
       " 2.0,\n",
       " 4.0,\n",
       " 17.0,\n",
       " 1.625,\n",
       " 5.333333333333333,\n",
       " 17.0,\n",
       " 1.9]"
      ]
     },
     "execution_count": 20,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "data\\\n",
    "    .map(lambda row: row[sc_h] / row[sc_w]).take(10)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "colab": {
     "base_uri": "https://localhost:8080/",
     "height": 507
    },
    "id": "HckZvR72xs7W",
    "outputId": "2b49fccc-96e2-44b9-f2ae-726caa75b1b4"
   },
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "2708\n"
     ]
    },
    {
     "data": {
      "text/plain": [
       "<matplotlib.axes._subplots.AxesSubplot at 0x7f1412214a30>"
      ]
     },
     "execution_count": 21,
     "metadata": {},
     "output_type": "execute_result"
    },
    {
     "data": {
      "image/png": "iVBORw0KGgoAAAANSUhEUgAAA20AAAJLCAYAAAB0cH5VAAAABHNCSVQICAgIfAhkiAAAAAlwSFlzAAALEgAACxIB0t1+/AAAADh0RVh0U29mdHdhcmUAbWF0cGxvdGxpYiB2ZXJzaW9uMy4yLjIsIGh0dHA6Ly9tYXRwbG90bGliLm9yZy+WH4yJAAAgAElEQVR4nO3de5gkZ10v8O8vbEQxEMCsERLiIgQ1HiXgykU5D3iQ6ypBQQQeIaASjwcUAZUF9cEbuAcBD3gARUFQQYyiElmUS4h6EAO5EAIhRAIsJDGEcAdBhfCeP6pWOsPO9Mz0zPTbM5/P8/QzNVX963qrqqemv/3WpVprAQAAoE9HzbsBAAAALE9oAwAA6JjQBgAA0DGhDQAAoGNCGwAAQMeENgAAgI7tmncDkuS4445re/bsmXczAAAA5uKCCy74aGtt95GmdRHa9uzZk/PPP3/ezQAAAJiLqvrgctMcHgkAANAxoQ0AAKBjQhsAAEDHhDYAAICOCW0AAAAdE9oAAAA6JrQBAAB0TGgDAADomNAGAADQMaENAACgY0IbAABAx4Q2AACAjgltAAAAHRPaAAAAOia0AQAAdExoAwAA6JjQBgAA0DGhDQAAoGNCGwAAQMeENgAAgI4JbQAAAB0T2gAAADomtAEAAHRMaAMAAOjYrnk3YKPt2X9wxemHDuzbopYAAADMTk8bAABAx4Q2AACAjgltAAAAHRPaAAAAOia0AQAAdExoAwAA6JjQBgAA0DGhDQAAoGNCGwAAQMeENgAAgI4JbQAAAB0T2gAAADomtAEAAHRMaAMAAOiY0AYAANAxoQ0AAKBjQhsAAEDHhDYAAICOCW0AAAAdE9oAAAA6JrQBAAB0TGgDAADomNAGAADQMaENAACgY1NDW1XdqqrOqap3V9UlVfX4cfyvVNVVVXXR+Lj/RM1Tquryqrqsqu6zmQsAAACwne1axXO+mORJrbULq+rGSS6oqjeM0367tfasySdX1SlJHprk25LcMskbq+p2rbXrNrLhAAAAO8HUnrbW2tWttQvH4c8kuTTJCSuUnJbkla21/2itfSDJ5UnutBGNBQAA2GnWdE5bVe1Jcockbx1HPa6qLq6ql1TVzcZxJyS5YqLsyqwc8gAAAFjGqkNbVR2T5FVJfra19ukkL0xymySnJrk6ybPXMuOqOqOqzq+q86+99tq1lAIAAOwYqwptVXV0hsD28tbaXyZJa+2a1tp1rbUvJfn9fPkQyKuS3Gqi/MRx3PW01l7UWtvbWtu7e/fuWZYBAABg21rN1SMryYuTXNpae87E+FtMPO0Hk7xrHD4ryUOr6oZVdeskJyd528Y1GQAAYOdYzdUjvyfJI5K8s6ouGsc9NcnDqurUJC3JoSQ/mSSttUuq6swk785w5cnHunIkAADA+kwNba21NyepI0x67Qo1T0/y9BnaBQAAQNZ49UgAAAC2ltAGAADQMaENAACgY0IbAABAx4Q2AACAjgltAAAAHRPaAAAAOia0AQAAdExoAwAA6JjQBgAA0DGhDQAAoGNCGwAAQMeENgAAgI4JbQAAAB0T2gAAADomtAEAAHRMaAMAAOiY0AYAANAxoQ0AAKBjQhsAAEDHhDYAAICOCW0AAAAdE9oAAAA6JrQBAAB0TGgDAADomNAGAADQsV3zbsCR7Nl/cMXphw7s26KWAAAAzJeeNgAAgI4JbQAAAB0T2gAAADomtAEAAHRMaAMAAOiY0AYAANAxoQ0AAKBjQhsAAEDHhDYAAICOCW0AAAAdE9oAAAA6JrQBAAB0TGgDAADomNAGAADQMaENAACgY0IbAABAx4Q2AACAjgltAAAAHRPaAAAAOia0AQAAdExoAwAA6JjQBgAA0DGhDQAAoGNCGwAAQMeENgAAgI4JbQAAAB0T2gAAADomtAEAAHRMaAMAAOiY0AYAANAxoQ0AAKBjQhsAAEDHhDYAAICOCW0AAAAdE9oAAAA6JrQBAAB0TGgDAADomNAGAADQMaENAACgY0IbAABAx4Q2AACAjgltAAAAHRPaAAAAOia0AQAAdExoAwAA6JjQBgAA0DGhDQAAoGNCGwAAQMeENgAAgI4JbQAAAB0T2gAAADomtAEAAHRMaAMAAOiY0AYAANAxoQ0AAKBjQhsAAEDHhDYAAICOCW0AAAAdE9oAAAA6JrQBAAB0TGgDAADomNAGAADQsamhrapuVVXnVNW7q+qSqnr8OP7mVfWGqnrv+PNm4/iqqudV1eVVdXFV3XGzFwIAAGC7Wk1P2xeTPKm1dkqSuyR5bFWdkmR/krNbaycnOXv8PUnul+Tk8XFGkhdueKsBAAB2iKmhrbV2dWvtwnH4M0kuTXJCktOSvGx82suSPHAcPi3JH7XBuUluWlW32PCWAwAA7ABrOqetqvYkuUOStyY5vrV29Tjpw0mOH4dPSHLFRNmV4zgAAADWaNWhraqOSfKqJD/bWvv05LTWWkvS1jLjqjqjqs6vqvOvvfbatZQCAADsGKsKbVV1dIbA9vLW2l+Oo685fNjj+PMj4/irktxqovzEcdz1tNZe1Frb21rbu3v37vW2HwAAYFtbzdUjK8mLk1zaWnvOxKSzkpw+Dp+e5NUT4x85XkXyLkk+NXEYJQAAAGuwaxXP+Z4kj0jyzqq6aBz31CQHkpxZVT+e5INJHjJOe22S+ye5PMnnkjx6Q1sMAACwg0wNba21NyepZSbf8wjPb0keO2O7AAAAyBqvHgkAAMDWEtoAAAA6JrQBAAB0TGgDAADomNAGAADQMaENAACgY0IbAABAx4Q2AACAjgltAAAAHRPaAAAAOia0AQAAdExoAwAA6JjQBgAA0DGhDQAAoGNCGwAAQMeENgAAgI4JbQAAAB0T2gAAADomtAEAAHRMaAMAAOiY0AYAANAxoQ0AAKBjQhsAAEDHhDYAAICOCW0AAAAdE9oAAAA6JrQBAAB0TGgDAADomNAGAADQMaENAACgY0IbAABAx4Q2AACAjgltAAAAHRPaAAAAOia0AQAAdExoAwAA6JjQBgAA0DGhDQAAoGNCGwAAQMeENgAAgI4JbQAAAB0T2gAAADomtAEAAHRMaAMAAOiY0AYAANAxoQ0AAKBjQhsAAEDHhDYAAICOCW0AAAAdE9oAAAA6JrQBAAB0TGgDAADomNAGAADQMaENAACgY0IbAABAx4Q2AACAjgltAAAAHRPaAAAAOia0AQAAdExoAwAA6JjQBgAA0DGhDQAAoGNCGwAAQMeENgAAgI4JbQAAAB0T2gAAADomtAEAAHRMaAMAAOiY0AYAANAxoQ0AAKBjQhsAAEDHhDYAAICOCW0AAAAdE9oAAAA6JrQBAAB0TGgDAADomNAGAADQMaENAACgY0IbAABAx4Q2AACAjgltAAAAHRPaAAAAOia0AQAAdExoAwAA6JjQBgAA0DGhDQAAoGNCGwAAQMeENgAAgI4JbQAAAB2bGtqq6iVV9ZGqetfEuF+pqquq6qLxcf+JaU+pqsur6rKqus9mNRwAAGAnWE1P20uT3PcI43+7tXbq+HhtklTVKUkemuTbxpoXVNUNNqqxAAAAO82uaU9orf1jVe1Z5eudluSVrbX/SPKBqro8yZ2S/PO6W7hA9uw/uOy0Qwf2bWFLAACA7WKWc9oeV1UXj4dP3mwcd0KSKyaec+U4DgAAgHVYb2h7YZLbJDk1ydVJnr3WF6iqM6rq/Ko6/9prr11nMwAAALa3dYW21to1rbXrWmtfSvL7GQ6BTJKrktxq4qknjuOO9Bovaq3tba3t3b1793qaAQAAsO2tK7RV1S0mfv3BJIevLHlWkodW1Q2r6tZJTk7yttmaCAAAsHNNvRBJVf1pknskOa6qrkzytCT3qKpTk7Qkh5L8ZJK01i6pqjOTvDvJF5M8trV23eY0HQAAYPtbzdUjH3aE0S9e4flPT/L0WRoFAADAYJarRwIAALDJhDYAAICOCW0AAAAdE9oAAAA6JrQBAAB0TGgDAADomNAGAADQMaENAACgY0IbAABAx4Q2AACAjgltAAAAHRPaAAAAOia0AQAAdExoAwAA6NiueTegJ3v2H1xx+qED+7aoJQAAAAM9bQAAAB0T2gAAADomtAEAAHRMaAMAAOiY0AYAANAxoQ0AAKBjQhsAAEDHhDYAAICOCW0AAAAdE9oAAAA6JrQBAAB0TGgDAADomNAGAADQMaENAACgY0IbAABAx4Q2AACAjgltAAAAHds17wYw2LP/4LLTDh3Yt4UtAQAAeqKnDQAAoGNCGwAAQMeENgAAgI4JbQAAAB0T2gAAADomtAEAAHRMaAMAAOiY0AYAANAxoQ0AAKBjQhsAAEDHhDYAAICOCW0AAAAdE9oAAAA6JrQBAAB0bNe8GwDrsWf/wRWnHzqwb4taAgAAm0tPGwAAQMeENgAAgI4JbQAAAB0T2gAAADomtAEAAHRMaAMAAOiY0AYAANAxoQ0AAKBjQhsAAEDHds27AczXnv0HV5x+6MC+LWoJAABwJHraAAAAOia0AQAAdExoAwAA6JjQBgAA0DGhDQAAoGNCGwAAQMeENgAAgI4JbQAAAB0T2gAAADomtAEAAHRMaAMAAOiY0AYAANAxoQ0AAKBjQhsAAEDHhDYAAICOCW0AAAAdE9oAAAA6JrQBAAB0TGgDAADomNAGAADQMaENAACgY0IbAABAx4Q2AACAjgltAAAAHRPaAAAAOia0AQAAdExoAwAA6JjQBgAA0DGhDQAAoGNCGwAAQMd2TXtCVb0kyfcn+Uhr7b+N426e5M+S7ElyKMlDWmufqKpK8twk90/yuSSPaq1duDlNpwd79h9cdtqhA/u2sCUAALA9raan7aVJ7rtk3P4kZ7fWTk5y9vh7ktwvycnj44wkL9yYZgIAAOxMU0Nba+0fk3x8yejTkrxsHH5ZkgdOjP+jNjg3yU2r6hYb1VgAAICdZr3ntB3fWrt6HP5wkuPH4ROSXDHxvCvHcQAAAKzD1HPapmmttapqa62rqjMyHEKZk046adZmAFzPSudbJs65BAAWx3p72q45fNjj+PMj4/irktxq4nknjuO+QmvtRa21va21vbt3715nMwAAALa39Ya2s5KcPg6fnuTVE+MfWYO7JPnUxGGUAAAArNFqLvn/p0nukeS4qroyydOSHEhyZlX9eJIPJnnI+PTXZrjc/+UZLvn/6E1oMwAAwI4xNbS11h62zKR7HuG5LcljZ20UAAAAg/UeHgkAAMAWENoAAAA6JrQBAAB0bOb7tDF/7kcFAADbl542AACAjgltAAAAHRPaAAAAOia0AQAAdExoAwAA6JjQBgAA0DGhDQAAoGNCGwAAQMeENgAAgI4JbQAAAB0T2gAAADomtAEAAHRMaAMAAOiY0AYAANAxoQ0AAKBjQhsAAEDHhDYAAICOCW0AAAAdE9oAAAA6JrQBAAB0TGgDAADomNAGAADQMaENAACgY0IbAABAx4Q2AACAjgltAAAAHRPaAAAAOia0AQAAdExoAwAA6JjQBgAA0DGhDQAAoGNCGwAAQMeENgAAgI4JbQAAAB0T2gAAADomtAEAAHRMaAMAAOiY0AYAANAxoQ0AAKBjQhsAAEDHds27AbBI9uw/uOL0Qwf2bVFLAADYKfS0AQAAdExoAwAA6JjQBgAA0DGhDQAAoGNCGwAAQMeENgAAgI4JbQAAAB0T2gAAADomtAEAAHRs17wbAADQoz37Dy477dCBfVvYEmCn09MGAADQMaENAACgY0IbAABAx5zTBgBMtdL5XYlzvAA2k542AACAjgltAAAAHRPaAAAAOuacNtgBnIsCALC49LQBAAB0TGgDAADomNAGAADQMaENAACgY0IbAABAx4Q2AACAjgltAAAAHRPaAAAAOia0AQAAdExoAwAA6NiueTeAnWvP/oPLTjt0YN8WtgQAAPqlpw0AAKBjQhsAAEDHhDYAAICOOacNtpDz+AAAWCs9bQAAAB0T2gAAADomtAEAAHRMaAMAAOiY0AYAANAxoQ0AAKBjQhsAAEDHhDYAAICOCW0AAAAdE9oAAAA6JrQBAAB0bNcsxVV1KMlnklyX5Iuttb1VdfMkf5ZkT5JDSR7SWvvEbM0EAADYmTaip+17W2unttb2jr/vT3J2a+3kJGePvwMAALAOm3F45GlJXjYOvyzJAzdhHgAAADvCrKGtJXl9VV1QVWeM445vrV09Dn84yfEzzgMAAGDHmumctiR3a61dVVVfn+QNVfWeyYmttVZV7UiFY8g7I0lOOumkGZsBwE60Z//BZacdOrBvC1sCAJtnpp621tpV48+PJPmrJHdKck1V3SJJxp8fWab2Ra21va21vbt3756lGQAAANvWukNbVX1tVd348HCSeyd5V5Kzkpw+Pu30JK+etZEAAAA71SyHRx6f5K+q6vDrvKK19ndVdV6SM6vqx5N8MMlDZm8mAADAzrTu0NZae3+S2x9h/MeS3HOWRgEAADDYjEv+AwAAsEGENgAAgI4JbQAAAB0T2gAAADomtAEAAHRMaAMAAOiY0AYAANCxWW6uDWyhPfsPLjvt0IF9W9gSAAC2kp42AACAjulpAwDYRhyZAduP0AZ0ywcPAACHRwIAAHRNaAMAAOiY0AYAANAx57QBLLHSuXSJ8+kAgK2lpw0AAKBjetrYcfSiAACwSPS0AQAAdExoAwAA6JjQBgAA0DHntAHAFnJeLQBrpacNAACgY3ragE2jRwEAYHZ62gAAADqmpw1gm1ipZ1OvJgAsLj1tAAAAHdPTBgDAzJzHDJtHTxsAAEDH9LQBK/LNKQDAfOlpAwAA6JjQBgAA0DGhDQAAoGPOaQMAYGE595qdQE8bAABAx4Q2AACAjgltAAAAHRPaAAAAOia0AQAAdExoAwAA6JjQBgAA0DGhDQAAoGNCGwAAQMd2zbsBAACwiPbsP7jstEMH9m1hS9ju9LQBAAB0TGgDAADomNAGAADQMaENAACgY0IbAABAx4Q2AACAjgltAAAAHRPaAAAAOia0AQAAdExoAwAA6JjQBgAA0DGhDQAAoGO75t0AAIDl7Nl/cMXphw7s26KWwMZa1Pf2Su3utc3bgZ42AACAjulpA2BHWtRvuQFYm+2wv9fTBgAA0DE9bQCwRtvhW1sAFoeeNgAAgI7paQMAgAWit3/n0dMGAADQMT1tAMyNb4vZrry318b6gpXpaQMAAOiYnjaADbQTvy3eics8Tyutb+v6K1lfa2N9bX+Lus9e1HZvFD1tAAAAHdPTBgBsqp3+DTnArPS0AQAAdExPGwBAR/RMAkvpaQMAAOiYnjYAAGDT6UVePz1tAAAAHdPTBtCRed0jybefALDxNur/up42AACAjulpAwBgx5rXEQ4sjh7eI3raAAAAOqanDQB2iB6+LQZg7fS0AQAAdExoAwAA6JjQBgAA0DGhDQAAoGNCGwAAQMeENgAAgI4JbQAAAB0T2gAAADomtAEAAHRMaAMAAOiY0AYAANCxTQttVXXfqrqsqi6vqv2bNR8AAIDtbFNCW1XdIMnzk9wvySlJHlZVp2zGvAAAALazzeppu1OSy1tr72+t/WeSVyY5bZPmBQAAsG1tVmg7IckVE79fOY4DAABgDaq1tvEvWvXgJPdtrf3E+Psjkty5tfa4ieeckeSM8ddvTnLZCi95XJKPrrM5i1g7z3lb5sWonee8LfNi1M5z3otYO895W+bFqJ3nvC3zYtTOc96WeTFqp9V/Y2tt9xGntNY2/JHkrkleN/H7U5I8ZYbXO38n1S5quy2z9WWZ+6ld1HZbX5a519pFbbdltr4scz+1s9Rv1uGR5yU5uapuXVVfleShSc7apHkBAABsW7s240Vba1+sqscleV2SGyR5SWvtks2YFwAAwHa2KaEtSVprr03y2g16uRftsNp5ztsyL0btPOdtmRejdp7zXsTaec7bMi9G7TznbZkXo3ae87bMi1G77vpNuRAJAAAAG2OzzmkDAABgAwhtAAAAHRPaAAAAOia0sbCq6viquuP4OH4DXu+YjWjXGuZ38xlqHzCPeVfVbavqQVV1yiqee9P1zGOiftfE8DFVtXet7a6qm8+ynuetqu44p/nepKq+s6puNod5H7eOmoXezvOyznW9IfvdGfd/q6qddR80vsaG/Z/ZimWeeP7cttO8/h57eG/Cpprl5nBb/UhyzBbP7+Yz1j9gq+ed5LZJHpTklFU896YzLt+uyW2TZO9a251kd5I7JPmO1W7fJKcmOTfJpUneOD7eM4674wzL86Ep0799nMcVGa78c7OJaW+bUvs9Y3svSXLnJG9I8r7xte46pfaHljwelOTDh39fxXL90sTwKUn+JckHkhxKcucpteckOW4cfsRY+wdJ3pnkp6fUfnHcNj++1vdakkcl+dg4v/sleX+Ss8f19bAptScleWWSa5O8N8nlST4yjtuzinnfanzu/0vy1CRHT0z76ym135Lkb5McTHKbJC9N8skkb0vyrVNq77jk8Z1Jrhz/PlZ8Xyf5sYnhE8d19ckkb0lyu1Us859MbOf7JPnQuO0+mOSHp9R+fHxP3DPjxa3WsJ3vN74X3zwu5yXj38WVSe65Wdt5Xtt41m01r3U91q97v5vZ9n+z7L9m2QfN9H9mjss8y3aaZb6z7ndn+buYyzJP1B2b5EeSPHF8/Mhq328Z9idPTvK88fHkrGI/sgG1627zAi/zPNfXuud9vddZa8E8H+n0Q/VYv+4P1rPsNLKAH6onlvONGXbs/5nkreMyvzTJsVNqLzrSeklylyTvmFL7xGUeT0ry8Sm1b05y3yQ3TfJz43vlNuO0t0+pfdv4/rxrko8muds4/o5J/mlK7ReSvCbJS5L84fj4zPjzJatY1xdODB9Mcr9x+E5J3jKl9l0Tw+cl+bpx+EZJLp5S+84k35/k5eN75dVJHprka1bR5ncmOS7JrZN8emI9H7+K+f5zhh3qDSbG3WCc97mrmPcbkvzPDB8EfifDh4bDyz1tO/9jkh9I8rAMgeehSWocd/aU2i+N8zpn4vH58eeb1rCNz0xyRoYjKX5w2nwPr++J4bdk/JA1boNpf1OXJXlckn9KclWS5ya5y7R5jrUXJfnW8e/iY4frxnEXTqld93ae1zaedVvNa11P1K93vzvL/m+W/dcs+6B1L++cl3mW7TTLfGfd787ydzGXZR6f98gMnxVfmOSXxsfvjuMeOaX2yWPb9yf50fGx//C4Taxdd5sXeJnnub7WPe+veK21PHkrHlnAD9Xj89b9wTo77EP1+Lxzk3zzxHK+bBx+TJK/mFL73hWmXT6l9t+T/HqSpx3h8ckpte9Y8vv3ZvhG8S6Z/gHz7RPDly63/Zep/a4MgfinJsZ9YNo6Xub99fbl2rVcu5OcMA6fk+Srx+EbJLlkDfP9miQPSfKX43vtFVNqL5oY/tcl06a9r1d6fyw77UjzHn//0Yz7kjVu58uXTJtW+6Ak/3D4738t23nJul7a/hW38ficS5LcZBx+c5KjJqetYd4nJfmFJBdm+CLnGWuovWKl7bCR23le23jWbTWvdb2K9T1tvzvL/m+W/dcs+6B1L++cl3mW7bRZ813NfneWv4u5LPP4nMtyhC+9k9wsyb9Mqf2XTPTyT4z/qlXsw2apXXebF3iZ57m+1j3vpY9Nu7n2DJ6R5Lcy9AItNe0cvBu31v5uHH5WVV2Q5O+q6hFJ2pTao1tr70ySqrq2tfbmJGmtXVhVX7OKdn93kgNJzmutvXB8nXu01h69itpJt2yt/e0477etYt5fqKoTWmtXJflskn8bx/9Hhg/WK9a21l6T5DXjfH4gQ2h7flW9rrX28BVqr2utfTTJR6vqs621941tvqaqpsw2yRAMLxtr3lZVvzsO/35VPXFK7d9W1cEkf5ShZy8ZDnd6ZJK/W7ZqcGGGw58uWDqhqn5iWqOr6tjW2qfGtp5TVQ9K8qok046Bn3zvPmXJtK9aqbC1dl5V3SvJT1fVORm+tZn2fp70TVV1VobegBOr6kattc+N046eUvuEJK+vqldl+FD7pqp6XZK7ZfhCYiX/9UZorX0+w7enZ1bVsUkeOKX2Q1X1m0lunOQ9VfXsDB+2vi/J1VNqL6iqFyR5Wa7//jg9Qwid5uiq+urW2r+Pbf+Tqvpwktcl+doptZN/c89ZMm3adn7VuG5/vap+LMMXVavdzidW1fMyrPPdVXV0a+0Lh5dnFfW/muScqnp+hl6cPx/fM9+b6X9Tk9v5Q0memeSZVfUtGb55X8knq+onk9wkySeq6gkZ3iffl2F/tpJZtvNctvFolm01r3WdzLbfXff+L7Ptv2bZB82yvMn8lnmWds8y31n3u7P8XcxrmTPWHWk//aVMvP+W8aUkt8zQYz/pFuO0zaqdpc2z1s9rmee5vmaZ9/X0GNoW7kP1OK9ZPljvtA/VSfK+qvrlJG/KcAjpRUlSVUdnSjhvrf1MVd0vyWlJThhHX5Xk+a21106Z76MzfMt6JHun1P7vDIcSnTvRlour6p5JfnlK7S8f3q6ttb8+PLKqbpPhH82KWmtfSvLcqvrzJP9n2vOXOG3J70eN8z4+Q3f/SvP9+6r67iQPz7CtL8jQW/nTrbX3TJnvy5d5zU9l+Me+kh9N8tgkn8pwGMF9MvxdfjDDobkreWSGQ35/NV9+f1yZ5G+SvHhKbTIcXnznDL1eh9v8xqr64Qwfklfy/Ko6prX22dbaCw6PrKrbZjgceEWttc8meUJV3SHDOlrtxXF+fmL4/LHuE1X1DUnOWsV8z6yqCzP0dN8uw/+GuyT509ba66aUn7PMa74nwzZYyekZDjX5UpJ7Zzjk8HUZtvNjptTOsp3nto0z27aa17qedb87y/5v3fuvzLAPmnF5kzkt84ztnmVdz7rfXfffxRyXOUmenuTCqnp9vhwYT0pyrwxH9azkZ5OcXVXvXVJ72wyHQW9W7SxtnrV+Xss8z/U1y7yvp8Yuum5U1Tcn+djYi7N02vGttWtWqH14kve31s5dMv6kJL/cWlv2H1MNV+N740RYOjz+Nkke1Fqb9o98suaWGT5Y722tfdMqnn/3JaMuaK19dtxpPLi19vwp9cdm+FB9+MPWlUlePe1DdVX9XGvtWdPat0ztTQ0Y8gAAAAk+SURBVDJ8qG5J/m+GD9WPzvAB4DdaaysGtxqu6vXUDOe2vSPJgdbaZ8Zl+dal2xB2mhq6rG/cWvv0vNsCwJHVcJXd++T6gfF1rbVPrKL2qAyniEzWntdau26Ta9fd5lnr57jM81xf65739azlWEoPj94fSc5Q2/+851j7/TOur3XXL2LtorZ7EWsXvN2LuC+w39262nn+XcxtO3t4bPRjoe7TVlVnLFrtPOe9iLUbUL+qE+rUzn3e86r9rhlqZ61fxNp5znun1c5z3rO2exH3Bfa7W1c7z7+LuW3nqnrRDLWvmVPtuts8a/0cl3me62tN8+7xnLaVLOLOap7zXsTaVdWPJ96fkOStbTgP6LClJ3ru+NpFbfeMtXdK0tpwrukpGa4q+57W2tOm1c5av4i1i9zuJa/1R621Ry5S7TznPWPt3TIc7vOu1trvbbfaqrpzhqs+frqGC3Xtz3A16XdnuGDatHmtu37G2p9J8lettSuWTlvFMs9S+1UZLmT2r204P/ThGS7QdmmS31ipdoPqvynD+fG3SnJdhiv2vWI175FZaqeYpX7qeaabVLsTl3me62tN8+7unLaVVNWjW2vTLq7RVe08572ItaupH/+xPDbDzvzUJI9vrb16nHZha+2Oauc/7znWPi3DvQN3Zbgf150zXMDhXhmOQX/6crWz1i9i7aK2u4aLN11vVIarXb4pSVprD+itdsHb/bbW2p3G4cdk+Pv8qwwXNfmb1tqBbVZ7SZLbt9a+OH6b/rkkf5Hhxua3b6390HK1s9bPWPupDFeRfl+SP03y5621a1dq6wbVvjzD3/GNMtwU+5gMFye7Z5K01h61WfXj/4vvz3APxftnuFrlJzPc4+1/tdb+fjNq+bKq+vrW2kfmMN+va60td3G57Wfp8ZI9PzLl5to91i5qu3teXxnuEXfMOLwnw5WmHj/+Pu1+LjuqdlHbvQG1N8jwz//T+fL9x74mq7uH4LrrF7F2Udud4UrDf5LkHknuPv68ehy+e4+1C97uyfuOnZdk9zj8tZm4Ofs2qr10YvjCJdNWc1+7ddfPWPv2DFdAvHeGqzZem+Gy96dnuLDRZtVePP7cleSajDfZzvDlwGr2Qeuuz7gfGYdvlOTvx+GTssr/F+upHZ93bIbbPb0nycczXJ360nHcV9zba7WPJH87ZfpNkvxmkj9O8vAl014wpfYbMlwZ8/lJvi7Jr4zr4cwkt1hF226+5PF1SQ5luG/ZzafU3nfJuntxkouTvCLJ8VNqDyQ5bhzem+H+lJdnOPrm7lNqL8xw9dzbrGNb7M3wZeKfZOiNfUOGK1ufl+QOq6g/JsmvZbjC+6fGv6tzkzxqrW3p7vDIqrp4uUkZbtzcXe08572ItRtQf1QbD5drrR2qqnsk+Yuq+sZMP7Ryp9Uuartnqf1iG67I9Lmqel8br77YWvt8Va3mniiz1C9i7aK2e2+Sxyf5xSQ/31q7qKo+31r7hyl186xd5HYfVcMV1I7KcJTOtUnSWvu3qjrSfVUXvfZdE0d9vKOq9rbWzq+q2yX5wpTaWetnqW1tuEXM6zPcDujoDL3ZD0vyrCS7N6n2qBoOcfzaDOHn2Awh5oZZ3f3OZq3fleHQxhtmvFVKa+1D4zJsZu2ZGXqr79Fa+3CS1HCbgtPHafderrCqljtipDIcYbKSP0zy3gy3tPqxGm5v9fDW2n9kuF3LSl6a5GCGdX1Ohltj3D/DrZ5+N195G4SlPpqvPE3hhAzBqCVZ6arpz8iX75337AxfHP1AhsNTfy8r325qX2tt/zj8W0l+pA2H1d8uQ+hb6bZNN0ty0wz3I/1whp7kP2ut/esKNYe9IMnTxvq3JHlCa+1eNdzu6QVJ7jql/uUZevjvk+QhGdb7K5P8UlXdrrX21FW0YbDWlLfZjwzfsJya5BuXPPZkONa5u9pFbfcCr683JTl1ybhdGe59c53aPuY9x9q3JrnROHzUxPhjs+Rb642uX8TaRW73+NwTk/x5hluPrKmXf161i9juDN+kvz/JB8aftxjHH5PpvT+LWHtshg+37xvfp18YX+MfMhyiOG19rbt+xtple4cO/61tUu0TxjZ+MMnPJDk7ye9n6MF52irW17rrM3wZcfH4/PckefQ4fneSf9ys2vF5l61n2jj9ugz/6845wuPzU2ovWvL7Lyb5pwy9XtP2u5M90B9a6XWXqX9ShuD17RPjPjCtbnzehcvNaxV/k5cm2TUOn7tk2rSe88n5/vcMYevD47pe8SqhU9bXanpj37Hk9/PGn0dlOH976nr7r9q1PHkrHhm6Su+2zLRX9Fi7qO1e4PV1YpJvWGba96jtY95zrL3hMuOPy8Q/mc2oX8TaRW73kpp9SZ6xlpp51y5yuyde50ZJbr1dazMchnb7JN+ZKYdvbXT9emqT3G6Gbbnu2rH+lkluOQ7fNMmDk9xpK+qTfNv4/G9ZR7tnqX19kl+Y3D4ZjhZ6coZ7/65U+64kJy8z7YoptZdm4ouucdyjMhyC98Epte+YGP6NJdNWDD8Tzzv85c9zktw4wz2SV1N3ZZInZgh+7894bY1x2rTDYH96XN//I8Mhnc/NcHj3ryb54ym1XxFkMxyaf98kfzil9p8z9Jj+cIYvFR44jr97kvNXscxvyfh5N8kDMpyvfXjaisF+6WOhLkQCAAA9GA+/3Z/hkMKvH0dfk+SsJAfaCjdfrqoHZwhJlx1h2gNba3+9Qu0zk7y+tfbGJePvm+R3Wmsnr1D7a0me2a5/VeZU1W3HNj94udojvNYDkjw1yZ7W2jes4vlPWzLqBa21a8dDSp/ZWnvklPp7JPmpJLfLcPTNFUn+OslLWmvLHvJcVa9srT10WvuWqb19kmcm+VKGHuGfynD461VJHtNae8uU+u9I8gdJTs4Qqn+stfYvVbU7ycNaa89bdVuENgAA2Di1gFfwXk9tDbeluE1r7V2L1O55166nXmgDAIANVFUfaq2dtFNq5znvRaxdT313V48EAIDeLeIVvF0tfetqN6J+ktAGAABrd3yGS7kvPXetMlyAYrvVznPei1i7EfX/RWgDAIC1e02SY1prFy2dUFV/vw1r5znvRazdiPovP985bQAAAP06at4NAAAAYHlCGwAAQMeENgAAgI4JbQAAAB0T2gAAADr2/wE1FL+O7FoK2QAAAABJRU5ErkJggg==\n",
      "text/plain": [
       "<Figure size 1080x720 with 1 Axes>"
      ]
     },
     "metadata": {
      "needs_background": "light"
     },
     "output_type": "display_data"
    }
   ],
   "source": [
    "# Выведите отсортированное распределение соотношений сторон экрана\n",
    "# в разрезе широкоформатные или нет (экран широкоформатный, если соотногшение >=16:9)\n",
    "tmp = data\\\n",
    "    .map(lambda row: (row[sc_h], row[sc_w]))\\\n",
    "    .filter(lambda row: row[0] > 0)\\\n",
    "    .filter(lambda row: row[1] > 0)\\\n",
    "    .map(lambda row: (round(row[0] / row[1], 1), 1))\n",
    "\n",
    "print(tmp.count())\n",
    "pd.Series(tmp.countByKey()).sort_index().plot.bar(figsize=(15, 10)) "
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "colab": {
     "base_uri": "https://localhost:8080/",
     "height": 511
    },
    "id": "yZgL4q1gyrRm",
    "outputId": "639f0a64-da98-433c-8661-695404c7017c"
   },
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "2705\n"
     ]
    },
    {
     "data": {
      "text/plain": [
       "<matplotlib.axes._subplots.AxesSubplot at 0x7f14099ecc10>"
      ]
     },
     "execution_count": 22,
     "metadata": {},
     "output_type": "execute_result"
    },
    {
     "data": {
      "image/png": "iVBORw0KGgoAAAANSUhEUgAAA20AAAJRCAYAAADWPV+oAAAABHNCSVQICAgIfAhkiAAAAAlwSFlzAAALEgAACxIB0t1+/AAAADh0RVh0U29mdHdhcmUAbWF0cGxvdGxpYiB2ZXJzaW9uMy4yLjIsIGh0dHA6Ly9tYXRwbG90bGliLm9yZy+WH4yJAAAgAElEQVR4nO3debhkd1kn8O8vNKCIQJY2RAI0YhBx1BhigiM8oCgGekbAQYRnRgIPTpwR3MatXWZwQ1tnXB+FeSJbUNkUkGiDgHHBLZCFGJaIidCRxEDCIuC4QfjNH+c0qb65Td9T99x7367+fJ6nnlv3VNV73/Orc6vOt86pc1rvPQAAANR0wk43AAAAwJEJbQAAAIUJbQAAAIUJbQAAAIUJbQAAAIUJbQAAAIXt2ukGkuSUU07pe/bs2ek2AAAAdsQVV1zxgd777vVuKxHa9uzZk8svv3yn2wAAANgRrbXrj3Sb3SMBAAAKE9oAAAAKE9oAAAAKE9oAAAAKE9oAAAAKE9oAAAAKE9oAAAAKE9oAAAAKE9oAAAAKE9oAAAAKE9oAAAAKE9oAAAAKE9oAAAAKE9oAAAAKE9oAAAAKE9oAAAAKE9oAAAAKE9oAAAAKE9oAAAAKE9oAAAAKE9oAAAAKE9oAAAAKE9oAAAAKE9oAAAAK27XTDbA69uw7cNT7HNy/dxs6AQCA1WFLGwAAQGFCGwAAQGFCGwAAQGFCGwAAQGFCGwAAQGFCGwAAQGFCGwAAQGFCGwAAQGFCGwAAQGFCGwAAQGFCGwAAQGFCGwAAQGFCGwAAQGFCGwAAQGFCGwAAQGFCGwAAQGFCGwAAQGFCGwAAQGFCGwAAQGFCGwAAQGFCGwAAQGG7droBdt6efQeOep+D+/duQycAAMBatrQBAAAUJrQBAAAUJrQBAAAUJrQBAAAUJrQBAAAUJrQBAAAUJrQBAAAUJrQBAAAUJrQBAAAUJrQBAAAUJrQBAAAUJrQBAAAUJrQBAAAUJrQBAAAUJrQBAAAUJrQBAAAUJrQBAAAUJrQBAAAUJrQBAAAUJrQBAAAUJrQBAAAUJrQBAAAUJrQBAAAUJrQBAAAUJrQBAAAUJrQBAAAUJrQBAAAUJrQBAAAUJrQBAAAUJrQBAAAUJrQBAAAUJrQBAAAUJrQBAAAUJrQBAAAUJrQBAAAUJrQBAAAUJrQBAAAUJrQBAAAUJrQBAAAUJrQBAAAUJrQBAAAUJrQBAAAUJrQBAAAUJrQBAAAUJrQBAAAUJrQBAAAUJrQBAAAUJrQBAAAUdtTQ1lq7d2vtj1pr72ytvaO19p3j9JNaa29srV07/jxxnN5aa7/cWruutXZ1a+2srZ4JAACAVbWRLW2fSPI9vfcHJXlIkme01h6UZF+SS3rvZyS5ZPw9SR6d5IzxckGS587eNQAAwHHiqKGt935T7/3K8frHklyT5F5JHpvkovFuFyV53Hj9sUle3AeXJrlHa+202TsHAAA4Dkz6TltrbU+SL0vy5iSn9t5vGm96X5JTx+v3SvLehYfdME4DAABgog2HttbaXZO8Msl39d4/unhb770n6VP+cGvtgtba5a21y2+55ZYpDwUAADhubCi0tdbumCGw/Wbv/VXj5Pcf2u1x/HnzOP3GJPdeePjp47TD9N4v7L2f3Xs/e/fu3cv2DwAAsNI2cvTIluT5Sa7pvf/8wk0XJzl/vH5+ktcsTH/KeBTJhyT5yMJulAAAAEywawP3+cok35zkba21q8ZpP5Rkf5JXtNaenuT6JE8cb3ttksckuS7JPyV52qwdAwAAHEeOGtp673+WpB3h5keuc/+e5Bmb7Ivj2J59B456n4P7925DJwAAsPMmHT0SAACA7bWR3SOZka1IAADAFLa0AQAAFCa0AQAAFCa0AQAAFCa0AQAAFCa0AQAAFCa0AQAAFOaQ/6wsp1cAAGAV2NIGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQ2K6dboDl7dl34Kj3Obh/7zZ0AgAAbBVb2gAAAAoT2gAAAAoT2gAAAAoT2gAAAAoT2gAAAAoT2gAAAAoT2gAAAAoT2gAAAAoT2gAAAAoT2gAAAAoT2gAAAAoT2gAAAAoT2gAAAAoT2gAAAAoT2gAAAAoT2gAAAAoT2gAAAAoT2gAAAAoT2gAAAAoT2gAAAAoT2gAAAArbtdMNwLFgz74DR73Pwf17t6ETAACON7a0AQAAFCa0AQAAFCa0AQAAFCa0AQAAFCa0AQAAFCa0AQAAFCa0AQAAFCa0AQAAFCa0AQAAFCa0AQAAFCa0AQAAFCa0AQAAFCa0AQAAFCa0AQAAFCa0AQAAFCa0AQAAFCa0AQAAFLZrpxuA48mefQeOep+D+/duQycAABwrbGkDAAAoTGgDAAAoTGgDAAAoTGgDAAAoTGgDAAAoTGgDAAAoTGgDAAAoTGgDAAAoTGgDAAAoTGgDAAAoTGgDAAAo7KihrbX2gtbaza21ty9M+9HW2o2ttavGy2MWbvvB1tp1rbV3tda+bqsaBwAAOB5sZEvbi5Kct870X+i9nzleXpskrbUHJXlSki8aH/Oc1tod5moWAADgeHPU0NZ7f1OSD22w3mOTvKz3/q+99/ckuS7JOZvoDwAA4Li2me+0PbO1dvW4++SJ47R7JXnvwn1uGKfdTmvtgtba5a21y2+55ZZNtAEAALC6lg1tz01y/yRnJrkpyc9NLdB7v7D3fnbv/ezdu3cv2QYAAMBqWyq09d7f33u/tff+ySS/ltt2gbwxyb0X7nr6OA0AAIAlLBXaWmunLfz6+CSHjix5cZIntdbu3Fq7X5Izkrxlcy0CAAAcv3Yd7Q6ttZcmeUSSU1prNyR5VpJHtNbOTNKTHEzyrUnSe39Ha+0VSd6Z5BNJntF7v3VrWgcAAFh9Rw1tvfcnrzP5+Z/m/s9O8uzNNAUAAMBgM0ePBAAAYIsJbQAAAIUJbQAAAIUJbQAAAIUJbQAAAIUJbQAAAIUJbQAAAIUJbQAAAIUJbQAAAIUJbQAAAIUJbQAAAIUJbQAAAIUJbQAAAIUJbQAAAIUJbQAAAIUJbQAAAIUJbQAAAIUJbQAAAIUJbQAAAIUJbQAAAIUJbQAAAIXt2ukGgOn27Dtw1Psc3L93GzoBAGCr2dIGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQ2K6dbuBYsWffgaPe5+D+vdvQCQAAcDyxpQ0AAKAwoQ0AAKAwoQ0AAKAwoQ0AAKAwoQ0AAKAwoQ0AAKAwoQ0AAKAwoQ0AAKAwoQ0AAKAwoQ0AAKCwXTvdALBz9uw7cNT7HNy/dxs6AQDgSGxpAwAAKExoAwAAKExoAwAAKExoAwAAKExoAwAAKExoAwAAKExoAwAAKExoAwAAKExoAwAAKExoAwAAKExoAwAAKExoAwAAKExoAwAAKExoAwAAKExoAwAAKExoAwAAKExoAwAAKExoAwAAKExoAwAAKExoAwAAKExoAwAAKExoAwAAKExoAwAAKExoAwAAKExoAwAAKExoAwAAKGzXTjcAHPv27Dtw1Psc3L93GzoBAFg9trQBAAAUJrQBAAAUJrQBAAAUJrQBAAAUJrQBAAAUJrQBAAAUJrQBAAAUdtTQ1lp7QWvt5tba2xemndRae2Nr7drx54nj9NZa++XW2nWttatba2dtZfMAAACrbiNb2l6U5Lw10/YluaT3fkaSS8bfk+TRSc4YLxckee48bQIAAByfjhraeu9vSvKhNZMfm+Si8fpFSR63MP3FfXBpknu01k6bq1kAAIDjzbLfaTu1937TeP19SU4dr98ryXsX7nfDOO12WmsXtNYub61dfssttyzZBgAAwGrb9IFIeu89SV/icRf23s/uvZ+9e/fuzbYBAACwkpYNbe8/tNvj+PPmcfqNSe69cL/Tx2kAAAAsYdnQdnGS88fr5yd5zcL0p4xHkXxIko8s7EYJAADARLuOdofW2kuTPCLJKa21G5I8K8n+JK9orT09yfVJnjje/bVJHpPkuiT/lORpW9AzAADAceOooa33/uQj3PTIde7bkzxjs00BAAAw2PSBSAAAANg6QhsAAEBhQhsAAEBhQhsAAEBhQhsAAEBhQhsAAEBhQhsAAEBhQhsAAEBhQhsAAEBhQhsAAEBhQhsAAEBhQhsAAEBhQhsAAEBhQhsAAEBhQhsAAEBhQhsAAEBhQhsAAEBhQhsAAEBhQhsAAEBhQhsAAEBhQhsAAEBhQhsAAEBhQhsAAEBhQhsAAEBhQhsAAEBhQhsAAEBhQhsAAEBhQhsAAEBhQhsAAEBhQhsAAEBhQhsAAEBhQhsAAEBhQhsAAEBhQhsAAEBhQhsAAEBhQhsAAEBhQhsAAEBhQhsAAEBhu3a6AYBD9uw7sKH7Hdy/d4s7AQCow5Y2AACAwoQ2AACAwoQ2AACAwoQ2AACAwoQ2AACAwoQ2AACAwoQ2AACAwoQ2AACAwlb+5NobOVmvE/UCAABV2dIGAABQmNAGAABQmNAGAABQmNAGAABQ2MofiAQ4PjkIEQCwKmxpAwAAKExoAwAAKExoAwAAKExoAwAAKExoAwAAKExoAwAAKExoAwAAKExoAwAAKExoAwAAKExoAwAAKExoAwAAKExoAwAAKExoAwAAKExoAwAAKExoAwAAKExoAwAAKExoAwAAKExoAwAAKExoAwAAKExoAwAAKExoAwAAKExoAwAAKExoAwAAKExoAwAAKExoAwAAKExoAwAAKExoAwAAKExoAwAAKExoAwAAKExoAwAAKExoAwAAKExoAwAAKExoAwAAKExoAwAAKExoAwAAKExoAwAAKExoAwAAKExoAwAAKExoAwAAKExoAwAAKExoAwAAKExoAwAAKExoAwAAKExoAwAAKExoAwAAKExoAwAAKExoAwAAKExoAwAAKGzXZh7cWjuY5GNJbk3yid772a21k5K8PMmeJAeTPLH3/uHNtQkAAHB8mmNL21f13s/svZ89/r4vySW99zOSXDL+DgAAwBK2YvfIxya5aLx+UZLHbcHfAAAAOC5sNrT1JG9orV3RWrtgnHZq7/2m8fr7kpy6yb8BAABw3NrUd9qSPLT3fmNr7XOSvLG19teLN/bee2utr/fAMeRdkCT3uc99NtkGAADAatrUlrbe+43jz5uTvDrJOUne31o7LUnGnzcf4bEX9t7P7r2fvXv37s20AQAAsLKWDm2ttc9qrX32oetJHpXk7UkuTnL+eLfzk7xms00CAAAcrzaze+SpSV7dWjtU5yW9999vrV2W5BWttacnuT7JEzffJgAAwPFp6dDWe393ki9dZ/oHkzxyM00BAAAw2IpD/gMAADAToQ0AAKAwoQ0AAKAwoQ0AAKAwoQ0AAKAwoQ0AAKCwzZynDWDl7dl34Kj3Obh/7zZ0AgAcr2xpAwAAKExoAwAAKExoAwAAKExoAwAAKExoAwAAKMzRIwGOIY5mCQDHH1vaAAAAChPaAAAACrN7JMA2sFsjALAsW9oAAAAKE9oAAAAKE9oAAAAK8502gOPQRr5jl/ieHQBUYEsbAABAYUIbAABAYUIbAABAYUIbAABAYUIbAABAYUIbAABAYUIbAABAYUIbAABAYUIbAABAYUIbAABAYUIbAABAYUIbAABAYbt2ugEAjm179h046n0O7t+7bXUAYNXY0gYAAFCY0AYAAFCY0AYAAFCY0AYAAFCY0AYAAFCY0AYAAFCYQ/4DsFKcggCAVWNLGwAAQGFCGwAAQGFCGwAAQGFCGwAAQGFCGwAAQGFCGwAAQGFCGwAAQGFCGwAAQGFCGwAAQGFCGwAAQGG7droBAFhle/Yd2ND9Du7fu8WdAHCssqUNAACgMKENAACgMKENAACgMKENAACgMKENAACgMKENAACgMKENAACgMKENAACgMCfXBoDjzEZO+O1k3wB12NIGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQ2K6dbgAA2Jg9+w4c9T4H9+/dhk5Wn7EGKrGlDQAAoDChDQAAoDChDQAAoDChDQAAoDAHIgEAluJgHUdnjIA52NIGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmKNHAgA7yhEWAT49W9oAAAAKE9oAAAAKs3skALASNrKbZWJXS+DYY0sbAABAYba0AQAwiYPHwPaypQ0AAKAwW9oAAIqzZQuOb7a0AQAAFFZ2S5tPlAAA5mX9Co5NtrQBAAAUVnZLGwAAUI8tttvPljYAAIDChDYAAIDC7B4JAAAju/5RkS1tAAAAhdnSBgDAMa/aFrJq/XBs27Itba2181pr72qtXdda27dVfwcAAGCVbcmWttbaHZL8apKvTXJDkstaaxf33t+5FX8PAIBjky1Sn96c41NtrOfqp1qduWslW7el7Zwk1/Xe3917/7ckL0vy2C36WwAAACur9d7nL9raE5Kc13v/lvH3b05ybu/9mQv3uSDJBeOvX5DkXUcpe0qSD8zQ3lx15qylzvbVUmf7aqmzfbXU2b5a6mxfLXW2r5Y621dLne2rdSzWuW/vffe6t/TeZ78keUKS5y38/s1JfmWTNS+fqbdZ6lTsaVXrVOxpVetU7GlV61TsaVXrVOxpVetU7GlV61TsaVXrVOxpVetU7KlKna3aPfLGJPde+P30cRoAAAATbFVouyzJGa21+7XW7pTkSUku3qK/BQAAsLK25OiRvfdPtNaemeT1Se6Q5AW993dssuyFm+9s1jpz1lJn+2qps3211Nm+WupsXy11tq+WOttXS53tq6XO9tVaqTpbciASAAAA5rFlJ9cGAABg84Q2AACAwoQ2AACAwoQ2AACAwrbk6JFzaK3dPcl5Se41Troxyet77/8wsU5Lcs6aOm/pE4/AMledsdZc81atzlxjPUs/sKpaa1+X5HE5/H/kNb33359QY1eSpyd5fJLPXayT5Pm9949vZ52x1qbna+Y6pcao2ljP2c9Y74FJHrump4t779dMrFNtOZprvkqNz5y1Co71SvYzZ61VXa6rvX8cVrPi0SNba09J8qwkb8htJ+U+PcnXJvmx3vuLN1jnUUmek+TaNXU+P8m39d7fsJ11xlpzzVu1OnON9Sz9LNQrs2KyBSs4ZV6gqo1RxbGecd5+MckDkrw4yQ3j5NOTPCXJtb3379xgnZcm+YckF62pc36Sk3rv37TNdeaar1nqjLWqjVG1sZ6ln7HWDyR5cpKXran1pCQv673v32CdUsvRjPNVanzmrFVwrFeynzlrrepyXfH94zC993KXJO9Kco91pp+Y5G8m1LkmyZ51pt8vyTXbXWfmeatWZ66xnqWf8TG/mOS1Gf75HzpenjRO+6UJdV6a5LlJHjL+w50+Xn9ukpdvd52Z520lx6joWM81b+v+HyRpGd5UNlXnaLdtd5255mtqnWNpjKqN9dR+Dt0/yR3XmX6nnXj+56wz13xVGp9VH+tV7KfqvFUao4rvH4uXqrtHtiTrbQL85HjbRu3Kbel20Y1J7rgDdZL55q1anbnGaK5+kuQxvfcH3O4PtPbyDC8UG/3E5MHr1LkhyaWttb+Z0M9cdZL55m1Vx6jiWM/V07+01r68937ZmulfnuRfJtT5UGvtG5O8svf+ySRprZ2Q5BuTfHgH6sw1X3PVSeqNUbWxnqufZHiN/9wk16+Zftp420ZVW47mmq9q4zNnrWpjvar9zFlrVZfriu8fn1I1tD07yZWttTckee847T4ZdpH7iQl1XpDkstbayxbq3DvDp+TP34E6yXzzVq3OXGM0Vz9JvRWTOf+Bq71AVRujimM9V09PTfLc1tpn57YPSu6d5CPjbRv1pCQ/k+Q5rbVDf/8eSf5ovG276zw188zXXHWSemNUbazn6idJvivJJa21a3P4a//nJ3nmhDpPTa3laK75qjY+c9aaq061MarWz5y1VnW5nqtOMu/rY5Ki32lLktbaiUm+Lrc/GMWkFa7W2hdm/S84vnMn6oy15pq3anXmGuu5+jkrw65n6/3zPaP3fsUG6+zJ8I/31blt5frQP96+3vt7trPOWGuueVvJMSo61rP1NNa7Zxb+R3rv75vy+DW1Tk6S3vsHl60xV5255mvO8RnrlRmjuepUW4bGDzHWHszqst77rUvUKrMczTVfFcdnzlqVxnpV+5mz1iov12XfP6qGNphLtRWTmeuUe4EqOEalxnrOnrZKa+2em32TmrNORdXGqNpYV+sHoIplXx+PufO0tdYunKnOj1aqM9aaa96q1fnRmeos1U/v/X299yvGy6ZWInrvH1xc0R5X5HeyzizztuJjVGqs5+xprdbalXPUyfTdvre0zlzzNeP4JMXGaK46BZehtNZ+b6Y6pZajGeer1PjMWavgWK9kP3PWWtXlusL7xzG3pa219uCN7o50lDr/sff+u1XqjLXmmrdqdeYa61n6GWtd2Xs/a4Y6B3rve6vUGWvNNW8rOUZFx3q2nmDVtNZO673ftNN9zG2u+VrV8ZlTtTGq1s+cLNdb55gLbQAkrbVTc/jumu9fokbL7b9L8JY+8Y1hrjpjrU3P18x1So1RtbGes5+FmiclSe/9Q5uoUWo5Gmtter7mqjPzfK3kWK94P6Xmba461ZbFuV8fS4a21trdk/xghpPZfk6GQ8DfnOEktPt77/+wwTrlTtQ747xVqzPXWM/Sz5qaZVZMZv8HLvQCVW2MKo71HD211s5M8n+T3D2Hn4D+HzKcyH5Du3C01h6V5DlJrl1T5/PHOm/Y5jpzzdcsdcZa1cao2ljP0s9Y6z5JfjbJI8c+WpK7JfnDDAfqObjBOqWWoxnnq9T4zFmr4FivZD9z1lrV5bri+8dh+hInd9vqS5LXJ/mBJPdcmHbPcdobJtSpeKLeueatWp25xnqWfsbHnZnk0gwn/v6D8fLX47SzJtR5VJLrkrwuyfPGy++P0x613XVmnreVHKOiYz3XvF2V5Nx1pj8kyV9NqHNNkj3rTL9fkmt2oM5c8zVLnaJjVG2sZ+lnfMxfJvmmJHdYmHaHDIfGvvRYXY5mnK9S47PiY72S/RSdt1JjNPNYz/b6+KnHLvOgrb4kedcyt61z31nORj5XnZnnrVqducZ6ln7G+5daMZnzH7jaC1S1MSo61nPN27Wf5rbrptRJsmud6XfaqTpzzdccdaqOUbWxnqOfDfR0xNuqL0fbNF/bPj7H8Vgfs/0cg/NW7TnbkfePxUvVk2tf31r7/iQX9XEXpHHXpKfmtpPvbUTFE/XONW/V6sw1RnP1kySf1Xt/89qJvfdLW2ufNaHOrtx2fq5FNya54w7USeabt1Udo4pjPVdPr2utHUjy4hx+IvunZNhyt1EvSHJZa+1la+o8KdOObDVXnbnma646ydaO0X0yfMJ8LI/1XP0kyRWtteckuWhNrfOTvHVCnWrL0VzzVW185qxVbaxXtZ85a63qcl3x/eNTqn6n7cQk+zKcqPnUDN9ren+Si5P8TN/glxTbbSez/aoM+6Mm85yot2XY33VSnbHWXPNWrc6ezDPWh/r5+rGfLNPPWOuXk9w/6//zvaf3/swN1vnBJE9Mst4/3it67z+9nXXGWnPN27EwRodWbo/1sZ6zp0dn/RPZv3ajNcY6X3iEOu+cWOdBGf5nN1vnMUeoM3W+ZhmfsdZcY1StzlxjPddzf6cM34u+3bxl+F70v06oNdf/x6brzDVfFcdnzlqVxnpV+5mz1iov1xXfPz5Vr2JoS5LW2gMzfC/q0t77Py5MP6/3vuG021o7N0MQ+dskD0zyFUneuczgj/VOHq/+Uu/9vyxTY029h2U4MMHb+rQvbZ+b5K977x9prd0lQ9A5K8k7kvxU7/0jG6zzHUle3XufuhVrbZ07JXlykr9PcmWS85J85djPhX3awVrun+QbMqzQ3prkXUle0nv/6BJ9reTK7Vir1ApuwZXS2V4sq63gMk1r7XN67zfvdB+LWmsn96InVwegoGX2qdzqS5LvyLCi/jtJDiZ57MJtV06o86wMBwu4PMlPJ7kkyf9M8qYkPzyhzsXrXP7x0PWJ8/aWhevfkmHz77OS/HmGLVIbrfOOjPvKJrkwyS8keehY61UT6nwkQ9D60yT/PckpSz5nv5nk5eOY/HqSVyX55iQvyrCr45Tn/g1JfiTJXyT51STPTvLOJI/Y6WXzeLgk+Zyd7mFNPyfvdA+VLhm28u/P8B25DyX54Hh9f5J7zPQ3XjfhvncbX19/PcmT19z2nAl17pnhwEW/muTkJD+a5Ookr0hy2oQ6J61zOZjkxCQnTRyH89aM+/PGnl6S5NQJdfYfem1N8uAk787wfYfrkzx8Qp0rx9fGz9vk8/vlGfaC+I0MH469McMeEpcl+bIJde6a5MfH96OPJLklw3vuU5foaVeSb81woJ6rx8vrkvy3JHecabm+cMJ97zD28xNJ/v2a235kQp27JPn+JN+X5DMy7PZ1cYYj5t11k/Mz6Tv142O+ZOH6Hcfl6eIkP5XkLhNrPXNhub5/hnWrDyd5c5IvnlDnVUn+8wzj8XkZdkn7iXHZ/LUkbwXJ+t0AAAocSURBVE/yW1nn+8Sfps4JSZ6W5PeS/NX4f/eyTFwHqbZMW66377nfwN/Z8HvsYY+bs4kZZ+Zth570JHsyhK7vHH9/68Q6dxgXro8muds4/TOTXD2hzpUZ3twekeTh48+bxusPnzhvb124flmS3eP1z8qwtW2jda5Z7G/NbVdN6WdcSB+VYR/bWzLst3t+ks+eUOfq8eeuDLsz3mH8vU0c67ctPPYuSf54vH6fKc/9+JiVXLkd719qBTcrunI71iq1gpsjH2F1X6Yd8fWsI1wenOSmCXVeOT5vj8vwJvnKJHc+9DxMqPP7Sb59nI+rx3m89zjtNRPqfDLJe9ZcPj7+fPfEsb5y4frzkvxkkvsm+e4kvzOhztsWrv9Rki8frz8gyeUT6rwnyf9J8ndJ3jL28blLLNNvSfLoDHtHvDfJE8bpj0zylxPqvCbD941PT/I/MnwoekaG76f81MSe5joC8XqvaSdleJ28YUKd52V4/fquJFck+fn1losN1HlFkp/LcOjvS5L8SpKHJfnfSX59Qp2PZViP+djC5dZD05dcpn8uwwerD8/wwe+LJz5n71i4fiDJ48frj0jy5xPq3JjktzO8T78iw6mD7rTEcv2mDB8+78sQ1r43w2vI05P84YQ6L8zwnvrQJL+Y4XX7azMcOfjbj9Vl2nK9fc/9WGuW99jDai7zoK2+LL4QjL/fNcMb+s9nYiBZ7/r4+5Q6J2R4c3xjkjPHaZPe/Bdq/VWGFeKTs+bNem2PR6nzW0metrCQnT1ef0CSyybUWRv47phh962XJrllQp23ZzgizonjP9tJ4/TPyLQj470tt63wnbg4RknePnGsV3Lldrx/qRXcrOjK7Vir1Apu5jvi660ZzoXzR+tc/nlCnavW/P7DGfYcOHnKcp3DX6//7tP9jaPU+Z7x/+OLF6a9Z8nn/soj9TCxp2ty254Rl665bcqHdYv9PCzDCtP7xufsgpnGesr70F+t+f2y8ecJGXbfnzLWcx2B+NYMH/YsvqYd+v3fJtS5euH6rgx7tLwqyZ0njtFV4882Pldt4fcpH2j+cobv1Z66MO09U8Z4nef+qoxbfKb2Mz7mXQvXL1tz25R5e+v4824Z9tB5bYYPtV6YaadDmWu5vnrN75eOP++caeszpZbptfNmud66537hedv0e+xhNZd50FZfxpk8c820XeMTe+uEOm/OuFk0yQkL0++eiSvJ4+NOzxCWfmXtC8KEGgcX/tnenXGrSIZgOmUl4O4ZPkn423E+Pz7W+5MkXzqhzhH/STNtk/J3j3//+gy7OF6SYdeEtyV51oQ635khhPxahnNhHQqmu5O8aeJYr+TK7drnbe2yOHE5mmUFNyu6cruBsd72FdwMuw9/fw5/kzs1Q3D/gwl13p7kjCPc9t6Jz9kJa6Y9NcMWxeuXGZ8kP7nscz/e/9Br9c8n+ews/yHbDRkC9veMr29t4bYpKwLfPj5vX53hU9xfyvAp8I9l2qfSt3udyLA3yXlJXjihzl9m2LviGzO8Zj9unP7wTPtw5C+SPHS8/vVJXr9w29RTtFw69rP4Xn1ChoMQvXlCnWuT3OcIt01Zrm/3P5nbvsow5VDkVy1cf8Ga26ae9+nBGd6LvmMcm8nL9bgcf0OS/5Q1K6FL9PPsDOshn5fkhzJsvblvxt3LJtRZb7k+OcNuhFO2kF2R4QO+c5J8ILd9mP35E/9fr0hy//H6WVlY98hwTIRjcpke77/Ky/XjN7tcz/Xcj/ef5T32sMct86CtvmR4w73nEW77ygl17nyE6adkwv7W6zx+bybu+rGBmndJcr8lHne3JF86LvQb3g1t4fEPmHEePjfj1owMR458QpJzlqjzReNjH7jJflZy5XZ8XKkV3Kzoyu34uFIruBm2QP9Mhg81Ppxhl6JrxmlTdml9QpIvOMJtj5tQ52eTfM0608/LtJWAH88634XIsML121OXyYVxvjTJ+5Z8/LPWXA7tzn7PTN+V7BEZvvf71gwfZr02yQWZ8N2WJC9bZj7WqfOlGfZEeF2GA3T9UoZdft+RNd9z2UCdt4zL4Z8dWp4yfMj2HRN72jOOz81J/ma83DxO2/B7Y5Jn5AgfXGbarm2/kYXdvhemf0uSj0+o87wjLNf3T/JnSzx3J2RYuf3TJH+/xONfuOZy6sIyfckS9Z6a4YPjD2TYy+adGb5HdPcJNSZ9IPtp6jwyw/EQrsmwe9srMwSem7NwbIQN1PnqDHtpXJvhA/Zzx+m7k/zsEsv0LePyfKiXHVmmx/uv6nL9ojmW64Xn/rrxuX/IMs/9+JhZ3mMPe9wyD3JxORYuOXzl9kM5fOX2xAl1Sq3cjo8ptYKb7Vm5vd1JKj9NjVlWbsdac63gfkkOX8F9wDh9mRXcByb5mrXLwHpvxhuo88gtrPPone4nw3eY/90ydbZpjHaqzhfOWGfTy+L4mHMzbCU5OcPRh783yWOWqHNObtu1+kEZPlCqVmdvFj7cWqLOw5L8ryX7OXeO+Vqnpy/K8MHdTo71uWv6WXYZ+oq5xmiscfJ4+Y1la6ypN+l9dTtqHaozdbleU+O0JB/cyfnKsEvlKZuts07dh47L0YZ3+V17KXvIf9hKrbWn9d5fuGp1NlurtfaZGXYNeHu1eatWZ6d6Gk/T8YwMH0CcmeEgTa8Zb7uy937WNtf59gxHkatSZ5b5qtjTzP18W4YPtHa8znj/Z2X47uiuDN8fPyfJH2c4CMDre+/PXrLOuRl2i161OnONz1J1trinlRij1trF60z+6gy7Aqb3/vVL1mkZzok7qc6ctbawTjLPGO1onbHWW3rv54zX/2uG94BXZ9hz53d77/s3WutT5kiPLi7H2iVLfiexep2KPa1qnZ3qKfMeXVedY6inVa2zUGuOoz2rsw11KvZUsM4sRx7PsNfJbEcwr9RTtTGae6wXri99tPjFy67AimqtXX2kmzJ8t+2YrFOxp1WtU7SnE3rv/5gkvfeDrbVHJPnt1tp9x1rqzFOnYk+rWidJPtF7vzXJP7XW/rb3/tGx7j+31j6pTrk6FXuqVufsDAdX++Ek39d7v6q19s+99z+ZUCMZjlkwR505a81Vp9oYzTnWJ7TWTszwfb3We78lSXrv/6+19okl6gltrLRTk3xdhu8QLWoZDgpxrNap2NOq1qnY0/tba2f23q9Kkt77P7bW/kOGk8p+sTqz1anY06rWSZJ/a63dpff+TxlWnJIkrbW7Zzg1iTq16lTsqVSd3vsnk/xCa+23xp/vzxLr3XPVqdjTqtYZ3T3D0Shbkt5aO633flNr7a6Z/qHWpxp0cVnJS4aThT/0CLe95FitU7GnVa1TsafMd3RddY6xnla1znj/WY72rM721KnYU7U66zx+liOPz1WnYk+rWmdNzaWOFt+7A5EAAACUdsJONwAAAMCRCW0AAACFCW0AAACFCW0AAACFCW0AAACF/X/vFLkG8kfY7AAAAABJRU5ErkJggg==\n",
      "text/plain": [
       "<Figure size 1080x720 with 1 Axes>"
      ]
     },
     "metadata": {
      "needs_background": "light"
     },
     "output_type": "display_data"
    }
   ],
   "source": [
    "# Выведите отсортированное распределение плотности пикселей\n",
    "# точек на дюйм (ppi)\n",
    "sc_h = train_header.index('sc_h')\n",
    "sc_w = train_header.index('sc_w')\n",
    "px_height = train_header.index('px_height')\n",
    "px_width = train_header.index('px_width')\n",
    "\n",
    "def calc_ppi(sc_h, sc_w, px_h, px_w):\n",
    "    px_diag = (px_h ** 2 + px_w ** 2) ** (1/2)\n",
    "    sc_diag = (sc_h ** 2 + sc_w ** 2) ** (1/2)\n",
    "    return px_diag / sc_diag\n",
    "\n",
    "tmp = data.map(lambda row: (row[sc_h], row[sc_w], row[px_height], row[px_width],))\\\n",
    "    .filter(lambda row: row[0] > 0)\\\n",
    "    .filter(lambda row: row[1] > 0)\\\n",
    "    .filter(lambda row: row[2] > 0)\\\n",
    "    .filter(lambda row: row[3] > 0)\\\n",
    "    .map(lambda row: (round(calc_ppi(*row) / 10, 0) * 10, 1))\n",
    "\n",
    "print(tmp.count())\n",
    "pd.Series(tmp.countByKey()).sort_index().plot.bar(figsize=(15, 10)) "
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {
    "id": "PNAGeyOmzEN4"
   },
   "source": [
    "# JOIN\n",
    "Повторите вышеописанное задание с помощью одной из функций ниже (отдельно рассчитайте для train и test, затем объедините результат)\n",
    "\n",
    "Пример для двух RDDs: (rdd = {(1, 2), (3, 4), (3, 6)} other = {(3, 9)})\n",
    "\n",
    "Имя функции |\tPurpose |\tExample |\tResult\n",
    "------------- |\t------- |\t------- |\t------\n",
    "subtractByKey |Remove elements with a key present in the other RDD.| rdd.subtractByKey(other) | {(1, 2)}\n",
    "join | Perform an inner join between two RDDs. | rdd.join(other) | {(3, (4, 9)), (3, (6, 9))}\n",
    "rightOuterJoin | Perform a join between two RDDs where the key must be present in the first RDD. | rdd.rightOuterJoin(other) | {(3,(Some(4),9)), (3,(Some(6),9))}leftOuterJoin | Perform a join between two RDDs where the key must be present in the other RDD. | rdd.leftOuterJoin(other) | {(1,(2,None)), (3,(4,Some(9))), (3,(6,Some(9)))}\n",
    "cogroup | Group data from both RDDs sharing the same key. | rdd.cogroup(other) | {(1,([2],[])), (3,([4, 6],[9]))}\n"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "id": "BkldcsRY6h6i"
   },
   "outputs": [],
   "source": [
    "sc_h = train_header.index('sc_h')\n",
    "sc_w = train_header.index('sc_w')\n",
    "px_height = train_header.index('px_height')\n",
    "px_width = train_header.index('px_width')\n",
    "\n",
    "def calc_ppi(sc_h, sc_w, px_h, px_w):\n",
    "    px_diag = (px_h ** 2 + px_w ** 2) ** (1/2)\n",
    "    sc_diag = (sc_h ** 2 + sc_w ** 2) ** (1/2)\n",
    "    return px_diag / sc_diag\n",
    "\n",
    "def get_rdd_ppi(rdd):\n",
    "    return rdd.map(lambda row: (row[sc_h], row[sc_w], row[px_height], row[px_width],))\\\n",
    "    .filter(lambda row: row[0] > 0)\\\n",
    "    .filter(lambda row: row[1] > 0)\\\n",
    "    .filter(lambda row: row[2] > 0)\\\n",
    "    .filter(lambda row: row[3] > 0)\\\n",
    "    .map(lambda row: (round(calc_ppi(*row) / 10, 0) * 10, 1))\\\n",
    "    .reduceByKey(lambda a, b: a + b)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "id": "GrQ4wE9N5vmS"
   },
   "outputs": [],
   "source": [
    "train_tmp = get_rdd_ppi(train)\n",
    "test_tmp = get_rdd_ppi(test)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "colab": {
     "base_uri": "https://localhost:8080/"
    },
    "id": "TOJn5koo843A",
    "outputId": "22901e2a-76c6-43c6-998d-bb4ed52ffff4"
   },
   "outputs": [
    {
     "data": {
      "text/plain": [
       "[(70.0, (147, 77)),\n",
       " (130.0, (102, 36)),\n",
       " (190.0, (29, 12)),\n",
       " (120.0, (102, 55)),\n",
       " (210.0, (30, 9)),\n",
       " (110.0, (118, 47)),\n",
       " (80.0, (161, 73)),\n",
       " (50.0, (140, 57)),\n",
       " (170.0, (49, 25)),\n",
       " (60.0, (121, 54))]"
      ]
     },
     "execution_count": 25,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "train_tmp.join(test_tmp).take(10)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "colab": {
     "base_uri": "https://localhost:8080/",
     "height": 493
    },
    "id": "jU01igJS7UeM",
    "outputId": "1332d04c-cbe6-43ed-c9c7-93631a607f7c"
   },
   "outputs": [
    {
     "data": {
      "text/plain": [
       "<matplotlib.axes._subplots.AxesSubplot at 0x7f140987fe80>"
      ]
     },
     "execution_count": 26,
     "metadata": {},
     "output_type": "execute_result"
    },
    {
     "data": {
      "image/png": "iVBORw0KGgoAAAANSUhEUgAAA20AAAJRCAYAAADWPV+oAAAABHNCSVQICAgIfAhkiAAAAAlwSFlzAAALEgAACxIB0t1+/AAAADh0RVh0U29mdHdhcmUAbWF0cGxvdGxpYiB2ZXJzaW9uMy4yLjIsIGh0dHA6Ly9tYXRwbG90bGliLm9yZy+WH4yJAAAgAElEQVR4nO3dfbRld10f/vc3DNgCggmMAQk4FEMBa4kQHlq1UqkYmFawKoW1WoJLVrqWPKg/f61j7a9YK3a0VattoY08GGrlwVJLdEChqZRqDeSBGB4CJMKkJAUSRFF/WpXw7R97jzlzc4fM3H2+937OzOu11lmz7z7nfs5nvnvfc/b77H32br33AAAAUNNZe90AAAAAJya0AQAAFCa0AQAAFCa0AQAAFCa0AQAAFCa0AQAAFLZvrxtIkgc+8IH9wIEDe90GAADAnrjmmms+1Xvfv919JULbgQMHcvXVV+91GwAAAHuitXbzie5zeCQAAEBhQhsAAEBhQhsAAEBhQhsAAEBhQhsAAEBhQhsAAEBhQhsAAEBhQhsAAEBhQhsAAEBhQhsAAEBhQhsAAEBhQhsAAEBhQhsAAEBhQhsAAEBhQhsAAEBhQhsAAEBhQhsAAEBhQhsAAEBhQhsAAEBhQhsAAEBhQhsAAEBhQhsAAEBhQhsAAEBhQhsAAEBh+/a6AdipA4eOnPRjjx4+OLATAAAYx542AACAwoQ2AACAwoQ2AACAwoQ2AACAwoQ2AACAwoQ2AACAwoQ2AACAwoQ2AACAwoQ2AACAwoQ2AACAwoQ2AACAwoQ2AACAwoQ2AACAwoQ2AACAwoQ2AACAwoQ2AACAwoQ2AACAwoQ2AACAwoQ2AACAwoQ2AACAwoQ2AACAwvbtdQOc3g4cOnLSjz16+ODATgAAYDPZ0wYAAFCY0AYAAFCY0AYAAFCY0AYAAFCY0AYAAFCY0AYAAFCY0AYAAFCY0AYAAFCY0AYAAFCY0AYAAFCY0AYAAFCY0AYAAFCY0AYAAFCY0AYAAFCY0AYAAFCY0AYAAFCY0AYAAFCY0AYAAFCY0AYAAFCY0AYAAFCY0AYAAFCY0AYAAFCY0AYAAFCY0AYAAFCY0AYAAFCY0AYAAFCY0AYAAFCY0AYAAFCY0AYAAFCY0AYAAFCY0AYAAFCY0AYAAFCY0AYAAFCY0AYAAFCY0AYAAFCY0AYAAFCY0AYAAFCY0AYAAFCY0AYAAFCY0AYAAFCY0AYAAFCY0AYAAFCY0AYAAFCY0AYAAFCY0AYAAFCY0AYAAFCY0AYAAFCY0AYAAFDY3Ya21tpDW2u/2lr7QGvt/a2175znn9Nae3tr7cb537Pn+a219lOttZtaa9e31h43+j8BAABwujqZPW2fTfI9vffHJHlykhe21h6T5FCSK3rv5ye5Yv45SZ6e5Pz5dkmSV6y9awAAgDPE3Ya23vvHe+/XztO/n+SGJA9J8swkl80PuyzJs+bpZyZ5bZ9cmeSLWmsPXnvnAAAAZ4BT+k5ba+1Akq9M8q4k5/bePz7f9Ykk587TD0nysZVfu2WeBwAAwCk66dDWWrtvkjcl+a7e+++t3td770n6qTxxa+2S1trVrbWrb7/99lP5VQAAgDPGSYW21to9MwW2/9h7/8/z7E8eO+xx/ve2ef6tSR668uvnzfOO03u/tPd+Ye/9wv379++0fwAAgNPayZw9siV5VZIbeu8/vnLX5UkunqcvTvLmlfnPm88i+eQkn1k5jBIAAIBTsO8kHvNVSf5ekve21q6b5/2jJIeTvLG19u1Jbk7y7Pm+tyR5RpKbkvxhkm9ba8cAAABnkLsNbb33X0vSTnD3U7d5fE/ywoV9wZ45cOjIKT3+6OGDgzoBAIBTPHskAAAAu+tkDo+kiFPZA2TvDwAAnB7saQMAAChMaAMAAChMaAMAAChMaAMAAChMaAMAAChMaAMAACjMKf9hF7lsAwAAp8qeNgAAgMKENgAAgMKENgAAgMKENgAAgMKENgAAgMKENgAAgMKENgAAgMKENgAAgMKENgAAgMKENgAAgMKENgAAgMKENgAAgMKENgAAgMKENgAAgMKENgAAgMKENgAAgMKENgAAgMKENgAAgMKENgAAgMKENgAAgMKENgAAgMKENgAAgMKENgAAgMKENgAAgMKENgAAgMKENgAAgMKENgAAgMKENgAAgMKENgAAgMKENgAAgMKENgAAgMKENgAAgMKENgAAgMKENgAAgMKENgAAgMKENgAAgMKENgAAgMKENgAAgMKENgAAgMKENgAAgMKENgAAgML27XUD7L0Dh46c0uOPHj44qBMAAGAre9oAAAAKE9oAAAAKE9oAAAAKE9oAAAAKE9oAAAAKE9oAAAAKE9oAAAAKE9oAAAAKE9oAAAAKE9oAAAAKE9oAAAAKE9oAAAAKE9oAAAAKE9oAAAAKE9oAAAAKE9oAAAAKE9oAAAAKE9oAAAAKE9oAAAAKE9oAAAAKE9oAAAAK27fXDQDLHTh05JQef/TwwUGdAACwbva0AQAAFCa0AQAAFCa0AQAAFCa0AQAAFCa0AQAAFCa0AQAAFCa0AQAAFCa0AQAAFCa0AQAAFCa0AQAAFCa0AQAAFCa0AQAAFCa0AQAAFCa0AQAAFCa0AQAAFCa0AQAAFCa0AQAAFLZvrxsAajtw6MhJP/bo4YMDOwEAODPZ0wYAAFCY0AYAAFCY0AYAAFCY0AYAAFCY0AYAAFCY0AYAAFCY0AYAAFCY0AYAAFCY0AYAAFCY0AYAAFCY0AYAAFDY3Ya21tqrW2u3tdbetzLvB1prt7bWrptvz1i57/taaze11j7UWvuGUY0DAACcCU5mT9vPJLlom/k/0Xu/YL69JUlaa49J8pwkXz7/zstba/dYV7MAAABnmrsNbb33dyb59EnWe2aS1/fe/7j3/tEkNyV54oL+AAAAzmhLvtP2otba9fPhk2fP8x6S5GMrj7llnncXrbVLWmtXt9auvv322xe0AQAAcPraaWh7RZJHJLkgyceT/NipFui9X9p7v7D3fuH+/ft32AYAAMDpbUehrff+yd77Hb33zyX56dx5COStSR668tDz5nkAAADswI5CW2vtwSs/flOSY2eWvDzJc1prX9Bae3iS85O8e1mLAAAAZ659d/eA1trrkjwlyQNba7ckeWmSp7TWLkjSkxxN8veTpPf+/tbaG5N8IMlnk7yw937HmNYBAABOf3cb2nrvz91m9qs+z+NfluRlS5oCAABgsuTskQAAAAwmtAEAABQmtAEAABQmtAEAABQmtAEAABQmtAEAABQmtAEAABQmtAEAABQmtAEAABQmtAEAABQmtAEAABQmtAEAABQmtAEAABQmtAEAABQmtAEAABQmtAEAABQmtAEAABQmtAEAABQmtAEAABQmtAEAABQmtAEAABS2b68bAM5cBw4dOenHHj18cGAnAAB12dMGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQ2L69buB0c+DQkZN+7NHDBwd2AgAAnA7saQMAAChMaAMAAChMaAMAAChMaAMAAChMaAMAAChMaAMAAChMaAMAAChMaAMAAChMaAMAAChMaAMAAChs3143ALBuBw4dOaXHHz18cFAnAADL2dMGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQ2L69bgBgkxw4dOSkH3v08MGBnQAAZwp72gAAAAoT2gAAAAoT2gAAAAoT2gAAAAoT2gAAAAoT2gAAAAoT2gAAAAq729DWWnt1a+221tr7Vuad01p7e2vtxvnfs+f5rbX2U621m1pr17fWHjeyeQAAgNPdyexp+5kkF22ZdyjJFb3385NcMf+cJE9Pcv58uyTJK9bTJgAAwJnpbkNb7/2dST69ZfYzk1w2T1+W5Fkr81/bJ1cm+aLW2oPX1SwAAMCZZqffaTu39/7xefoTSc6dpx+S5GMrj7tlnncXrbVLWmtXt9auvv3223fYBgAAwOlt8YlIeu89Sd/B713ae7+w937h/v37l7YBAABwWtppaPvkscMe539vm+ffmuShK487b54HAADADuw0tF2e5OJ5+uIkb16Z/7z5LJJPTvKZlcMoAQAAOEX77u4BrbXXJXlKkge21m5J8tIkh5O8sbX27UluTvLs+eFvSfKMJDcl+cMk3zagZwAAgDPG3Ya23vtzT3DXU7d5bE/ywqVNAQAAMFl8IhIAAADGEdoAAAAKE9oAAAAKE9oAAAAKE9oAAAAKE9oAAAAKE9oAAAAKE9oAAAAKE9oAAAAKE9oAAAAKE9oAAAAKE9oAAAAKE9oAAAAKE9oAAAAKE9oAAAAKE9oAAAAKE9oAAAAKE9oAAAAKE9oAAAAKE9oAAAAKE9oAAAAKE9oAAAAKE9oAAAAKE9oAAAAKE9oAAAAKE9oAAAAKE9oAAAAKE9oAAAAKE9oAAAAKE9oAAAAKE9oAAAAKE9oAAAAKE9oAAAAKE9oAAAAKE9oAAAAKE9oAAAAKE9oAAAAKE9oAAAAK27fXDQAwOXDoyEk/9ujhgwM7AQAqsacNAACgMKENAACgMKENAACgMKENAACgMKENAACgMKENAACgMKENAACgMKENAACgsDPy4tqncgHbxEVsAQCAvWNPGwAAQGFCGwAAQGFCGwAAQGFCGwAAQGFn5IlIAM4kp3LyJSdeAoB67GkDAAAoTGgDAAAoTGgDAAAoTGgDAAAoTGgDAAAoTGgDAAAoTGgDAAAoTGgDAAAoTGgDAAAoTGgDAAAoTGgDAAAoTGgDAAAoTGgDAAAoTGgDAAAoTGgDAAAoTGgDAAAoTGgDAAAoTGgDAAAoTGgDAAAoTGgDAAAoTGgDAAAoTGgDAAAoTGgDAAAoTGgDAAAoTGgDAAAoTGgDAAAoTGgDAAAoTGgDAAAoTGgDAAAoTGgDAAAoTGgDAAAoTGgDAAAoTGgDAAAoTGgDAAAoTGgDAAAoTGgDAAAoTGgDAAAoTGgDAAAoTGgDAAAoTGgDAAAoTGgDAAAoTGgDAAAoTGgDAAAoTGgDAAAoTGgDAAAoTGgDAAAoTGgDAAAoTGgDAAAobN+SX26tHU3y+0nuSPLZ3vuFrbVzkrwhyYEkR5M8u/f+O8vaBAAAODOtY0/bX++9X9B7v3D++VCSK3rv5ye5Yv4ZAACAHRhxeOQzk1w2T1+W5FkDngMAAOCMsDS09SRva61d01q7ZJ53bu/94/P0J5Kcu/A5AAAAzliLvtOW5Kt777e21r44ydtbax9cvbP33ltrfbtfnEPeJUnysIc9bGEbAAAAp6dFe9p677fO/96W5BeSPDHJJ1trD06S+d/bTvC7l/beL+y9X7h///4lbQAAAJy2dhzaWmv3aa194bHpJE9L8r4klye5eH7YxUnevLRJAACAM9WSwyPPTfILrbVjdX6u9/7LrbWrkryxtfbtSW5O8uzlbQIAAJyZdhzaeu8fSfLYbeb/dpKnLmkKAACAyYhT/gMAALAmQhsAAEBhQhsAAEBhQhsAAEBhQhsAAEBhQhsAAEBhS67TBsAZ7sChIyf92KOHDw7sBABOX/a0AQAAFCa0AQAAFCa0AQAAFCa0AQAAFCa0AQAAFObskQCcUZzxEoBNY08bAABAYUIbAABAYQ6PBKCcUzmEMXEYIwCnN3vaAAAAChPaAAAAChPaAAAACvOdNgBYE5cTAGAEe9oAAAAKE9oAAAAKE9oAAAAKE9oAAAAKE9oAAAAKE9oAAAAKE9oAAAAKE9oAAAAKE9oAAAAKE9oAAAAKE9oAAAAKE9oAAAAK27fXDQAAn9+BQ0dO6fFHDx8cUvtU6gKwPva0AQAAFCa0AQAAFCa0AQAAFCa0AQAAFCa0AQAAFCa0AQAAFOaU/wDAEKMuJ+AyBcCZxp42AACAwoQ2AACAwoQ2AACAwoQ2AACAwoQ2AACAwoQ2AACAwoQ2AACAwoQ2AACAwoQ2AACAwoQ2AACAwvbtdQMAAFUcOHTkpB979PDBgZ0A3MmeNgAAgMKENgAAgMKENgAAgMKENgAAgMKENgAAgMKENgAAgMKENgAAgMKENgAAgMJcXBsAYIO5IDic/uxpAwAAKExoAwAAKExoAwAAKExoAwAAKExoAwAAKExoAwAAKExoAwAAKExoAwAAKExoAwAAKExoAwAAKExoAwAAKExoAwAAKExoAwAAKExoAwAAKGzfXjcAAHC6O3DoyCk9/ujhg4M6YYlTWY6WIetkTxsAAEBhQhsAAEBhQhsAAEBhQhsAAEBhTkQCAMC2nHhjdxhn7o49bQAAAIUJbQAAAIUJbQAAAIUJbQAAAIUJbQAAAIU5eyQAALvqVM6WmDhjItjTBgAAUJjQBgAAUJjDIwEAOG24UDWnI3vaAAAACrOnDQAAKMPe0ruypw0AAKAwe9oAAOA0Za/V6cGeNgAAgMJK72nzyQAAANTjAum7y542AACAwkrvaQMAAKhu9J5He9oAAAAKE9oAAAAKc3gkAABwRtjUEx3a0wYAAFCY0AYAAFDYsNDWWruotfah1tpNrbVDo54HAADgdDYktLXW7pHk3yZ5epLHJHlua+0xI54LAADgdDZqT9sTk9zUe/9I7/1Pkrw+yTMHPRcAAMBpq/Xe11+0tW9JclHv/QXzz38vyZN67y9aecwlSS6Zf/yLST50Ck/xwCSfWlO7u1F3ZO1N7HlkbT1vfu1N7HlkbT1vfu1N7HlkbT1vfu1N7HlkbT1vfu0qPX9p733/dnfs2Sn/e++XJrl0J7/bWru6937hmlsaVndk7U3seWRtPW9+7U3seWRtPW9+7U3seWRtPW9+7U3seWRtPW9+7U3oedThkbcmeejKz+fN8wAAADgFo0LbVUnOb609vLV2ryTPSXL5oOcCAAA4bQ05PLL3/tnW2ouS/EqSeyR5de/9/Wt8ih0dVrmHdUfW3sSeR9bW8+bX3sSeR9bW8+bX3sSeR9bW8+bX3sSeR9bW8+bXLt/zkBORAAAAsB7DLq4NAADAckIbAABAYUIbAABAYUIbAABAYXt2ce2T1Vq7f5KLkjxknnVrkl/pvf/uGmq3JE/cUvvdfeHZWUbVnWuPHI8htTd0GQ7rGdh7rbVvSPKsHP83/ube+y8vrLsvybcn+aYkX7JaO8mreu9/WrT2kPEYWXvUeFiGd6k7rOe5/qOSPDPH93157/2GhXU3bp2ea48ajyF159qj1r2NW4ZDx7ny2SNba89L8tIkb8udF+c+L8nXJ/mnvffXLqj9tCQvT3LjltpfluQ7eu9vq1R3rj1yPIbU3tBlOKznledY+4uFDY3dq21jcdv6G7NOt9b+VZJHJnltklvm2ecleV6SG3vv37mg59cl+d0kl22pfXGSc3rvf6da7cHjsXFjbRnuTs9z7e9N8twkr99S+zlJXt97P7zDupu6To8ajyF159pDxmMTl+HIcU7qh7YPJXnS1r0brbWzk7yr9/7IBbVvSPL03vvRLfMfnuQtvfdHV6o71xg5HkNqb+gyHNbzXGfUi4UNjd2rbWPx+NobtU631j683d/xvOf+w73383dS9/PVvrv79rL2XoxH5bG2DE++r3WMR5Iv3/oBTGvtXknev9O+N3mdzqDxGFH3WO0R47GJy3DkOCf1D49sSbZLlZ+b71tiX+7cCFh1a5J7FqybjB2PUbU3cRmO7DlJnnGCF4s3JPlwkp2GicdvU/eWJFfOLyRLjKo9aixG1x41Hpu4DJPNW6f/T2vtCb33q7bMf0KS/7OgbpJ8urX2rUne1Hv/XJK01s5K8q1Jfqdo7ZHjsYljbRkeb+R4fC7TXvSbt8x/8HzfTm3qOj1qPEbVTcaNxyYuw5HjXD60vSzJta21tyX52DzvYZkOU/tnC2u/OslVrbXXr9R+aKZdmK8qWDcZOx6jam/iMhzZczLuxcKGxu7VtrF4vE1bp5+f5BWttS/MnR/8PDTJZ+b7lnhOkh9J8vLW2rEevyjJr873Vaz9/Iwbj5G1R42HZXi8kePxXUmuaK3dmOPfb78syYsW1H1+NnOdHjUeo+om48ZjVN2RtUeOc+3DI5M/OyTtG3LXE0Is3chIa+3R2f7Lgh+oWHeuPXI8htTe0GU4sufHJXlFku1eLF7Ye79mh3UPZHpj/brcuUF77I31UO/9owt6HlJ71FjsQu0DGTMeQ+ruQu2NW6fn+g/Kyt947/0TS+ptU/8BSdJ7/+111h1Ve+R4bOpYW4Z3qT+i57Ny1xOKXdV7v2MNtTdunR41HiPHea4/ajw2ahkOXZ+rhzY4XQ1+IbKhsQu15/o2Fu+su5Hr9G5prT1o3evfbtTeRKPGwzI83ib2DJtqY6/T1lq7dGDtH9ikunPtkeMxpPaGLsO19dx7/0Tv/Zr5ttY3vd77b69u3M4b02VrDx6LYbXn+kPGetOW4Vx3I9fpVa21a0fUnS09RH7Xa48cjw0da8vweMPGo7X2S4PqbuQ6PXA8htSdaw8Zj01chusY543d09Zae/ySw5vupvbf6r3/4qbUnWuPHI8htTd0GQ7rea5/be/9cQPqHum9H1x33ZG1R43FLtQeNR4btwzn2hu3TgN7r7X24N77x/e6jypGjYdx3h3rGOeNDW0AsERr7dwcfzjnJ9dUt+Wu32l4d1/DG+7g2kPGY2TtUeNhGd6l7rCeV57jnCTpvX96jTU3bp1eqb/28Rhcd9S6t5HLcMj6XDm0tdbun+T7Ml2w9YsznYb9tkwXVj3ct1xH6xRrb+LFcUeOx5DaG7oMh/W85XnW/mJhQ2P3attY3Lb+RqzTrbULkvy7JPef6yXT9d9+N8l39N53fHhMa+1pSV6e5MYttb9srv22arUHj8fGjbVluDs9z7UfluRHkzx17rUluV+S/5bpZENHd1h3U9fpUeMxpO5ce8h4bOIyHDnOSZLee9lbkl9J8r1JHrQy70HzvLctrP26TGc7e3KmBXXePP2KJG+oVncXxmNI7Q1dhsN6nmtdkOTKJDck+a/z7YPzvMctqPu0JDcleWuSV863X57nPW1hz0NqjxqLXag9ajw2bhlu4jqd5LokT9pm/pOT/ObCsbghyYFt5j88yQ0Vaw8ej40ba8twV8fjN5L8nST3WJl3j0yXEriy6HiMrD1qPIbUHTkem7gMR45z7718aPvQTu47ydof3sl9e1V3F8ZjSO0NXYbDep5rjHqxsKGxe7VtLO7CWA8c5xs/z303LRyLG5Ps22b+varWHj0emzbWluHujsdO7tvr8dij2qPGY8d1R47HJi7DkePcey9/ce2bW2v/MMllfT7MZj785vm586J1O7WJF8cdOR6jam/iMhzZc5Lcp/f+rq0ze+9Xttbus6Duvtx5jaxVtya554K6I2uPGovRtUeNxyYuw2Tz1um3ttaOJHlt7vybfmiS52Xak7fEq5Nc1Vp7/Zbaz8nyM+2Nqj1yPDZxrLer+7BMn6Cfictw5Dp9TWvt5Uku21L74iTvWVB3U9fpUeMxqm4ybjw2cRmOHOfy32k7O8mhTBdPPjfTd4s+meTyJD/SF3y5r9150da/num402TMxXFbpmNmF9Wda48cjyG1N3QZHuv5G+ees66e5/o/leQR2f7F4qO99xftsO73JXl2ku3eWN/Ye//nC3oeUnvUWOxC7VHjsV3dYxuLJZfhXHsT1+mnZ3pdWv2u3OW997fstOZK7UefoPYH1lD7MZlem9Zau7X2jBPUXcd4bNxYD16Go3oeuQxHrXf3yvTd9LuMR6bvpv/xgtoj17shtUeNx8hxnuuPGo+NWobDx7lyaEuS1tqjMn1X6cre+x+szL+o974oabfWnpQpRPxWkkcl+StJPrCOlWGu/4B58id77393HTW31P+aTF/Qf29f8EXgudaTknyw9/6Z1tq9M4WWxyV5f5If7r1/Zod1X5LkF3rv69hDtbX2vZI8N8n/TnJtkouSfFWmni/ty0768ogkfzvTBuIdST6U5Od677+3tO+5/qgXOBuLx9e2sbh7tYcsx1HrHXuntfbFvffb9rqPU9Fae0Df4Au7A5uvdGibN/hfmOl7DRck+c7e+5vn+xZd+6e19tIkT890+M3bM4WfdyT5+iS/0nt/2Q7rXr7N7K/LdOaY9N6/cSd159rv7r0/cZ5+Qaax+S+Zvqz/i733wwtqvz/JY3vvn23TBaT//yRvynQGnMf23v/2Dut+Zq71W0l+LsnP994/tdM+t9T+j5mW359P8pkk90nyC3PPrfd+8Q7rviTJ30zyziTPyLRL+3cznaXyO3rv71jcPHvCxiLJcWeIXT0CYO1niN3med/ae3/6gt+/X6a+z0vylt7761bue3nv/Tt2WPdBSV6a5HNJ/kmSF2f60OqDmd53d3xtoTaf9nqLa5N8ZabX6SVHW/zZh7fzMv2xTO/l70vy3X2HZy9trR1O8i97759qrT0+yc9n+vDuXkme13v/7wt6vjbJf870IeBHdlpnm7pPyHTWulszrSOvTvKETN9Hu6T3vuNDs1pr903yD5N8c6Z1708yvaf/u977zyzs+9hZoJ+V4z+YWXzG7c/znJf23i9Z8Pv3SPKCTGPx1t77/1y57x/33n9oQe17J3lRptekf53pKItvzvS3+IOrOy+Waq19uPf+yDXU+cu99+vn6XtmOmnbsb/DH+q9/+EO674oyevnv8NHJHlNkq9I8uEkL+i9v3dBz2dlOmTxm3Pnh/MfzrROv2NB3bHL7+6+9LaXtyTvTXLfefpAkjG1PuwAAAk/SURBVKszvYEkyXvWUPseSe6d5PeS3G+e/+eTXL+g7rVJfjbJU5J87fzvx+fpr13Y83tWpq9Ksn+evk+mvW1Lat+wMn3tlvuuW9JzkrMyBctXJbk90/HCFyf5woU9Xz//uy/T4Yv3mH9uC5fhe1dq3TvJO+bphy1d7+Y6909yONOHEZ9O8tvz9OEkX7S0/gme860Lf/9+Sf55kv+Q5Llb7nv5groPynS2z3+b5AFJfiDJ9UnemOTBC3s+Z5vb0SRnJzlnYe2LtizPV859/1yScxfUPZzkgfP045N8JNNG181reP24Nsk/TvIXBqxfT8h0WPLPZnoDfHumDzquSvKVC+reN8kPZtp7/pn59ePKJM9f2O+JzhB7KMvPavu4E9wen+TjC2u/aV5HnpXpcJs3JfmCY8t3Qd1fzhTUDs3r8ffOy/HFSd68sOfPJfnoltufzv9+ZOk6vTL9yiQ/lORLk3x3kv+yoO57V6Z/NckT5ulHJrl6Yc8fTfIvk/yvJO+ee/2SJTXnuu/O9EH0czMdSvwt8/ynJvmNhbXfnOk73ecl+X+S/H9Jzs/0vZ0fXlh71Fmgt3v9PyfT+8wtC3t+ZabX+u9Kck2SH99undxh7Tdm+vDh5UmuSPJvknxNkn+R5D8sqPv7mbZ1f3/ldsex+Qt7Xv07/LEkP5Npm/cnkrx2Qd33r0wfSfJN8/RTkvz6wp5fk2l746uT/KtM7zNfn+nMxy+utvz+rP7SAiNvqwts/vm+md5cfjwLgsRc6z3bTc8/LwkpZ80vwm9PcsE8b9Eb00rt38y0wfmAbHnj2Pp/2EHtn0/ybfP0a5JcOE8/MslVC+puDYD3zHSo0+uS3L6w5/dl+uTz7PmF55x5/p/LsjPLvTd3bgidvTrWSd63huU4ZIMxNha31raxeHztIRuLc+0hG4wZtLGYsWe1vSPTkRW/us3tjxbWvm7Lz9+f5NczvScs+TtcfT/8X5/vOXdQ+3vmv/OvWF0X17Terf4dbh2bJe/jN2Q+W2K2nKY7yz8gXe35azJt3H1iXj8uGbQMl24f/OaWn6+a/z0r09cqltQedRboOzJ96LX6+n/s5z9Z2PP1K9P7klyaae/pF6xhrK+b/23zetFWfl7yYfRPZfrO8bkr8z66pNcTrHvXJbnnmnr+0Mr0VVvu23Hd7X7/2N/5vAwXnRF7xPL7s/rrWGCjbvMb3wVb5u2bV7w7FtZ+V5J7z9Nnrcy//5I3v5U652UKQv9m6wvogppHV150PpJ5T0SmMLv0jfX+mT4d+a15bP50fo7/nunwyJ3WPeEL2LHxX1D7u+ceb07ykkyfavx0ptD10gV1vzNTgPjpTLu0j4XZ/UneuYblOOryCjYWj/99G4sn7nltG4snsRx3vBGTQRuLSd6W6XCv1Q2YczN9aPBfF47F+5Kcf4L7Praw9g1Zeb+a5z0/057Im9cxzpkOZ1rbejfXOPZ++ONJvjDr+yDzlkxh/nvm94K2ct+SjcUXz+vI12X6NP4nM+05+KdZ+Gn5dq+XmY76uSjJaxbU/Y1MR7R8a6b3xGfN8782yz/w+Z9Jvnqe/sZMXyE5dt/SDzmunHte3Q47K9NhZe9aUPfGJA87wX1L/w7v8tqT6fDiX8/y0+dftzL96i33Lb1UzeMzbSO8ZB7jdf0dfiTT4dTfnC2BZ0nPSV6Wabv0LyT5R5n2bH5pkm9L8ksLe74mySPm6cdlZdsu07ktyi2/3uuHtvOysjdiy31ftbD2F5xg/gOzspG3hv/DwSw8fOAknuPeSR6+plr3S/LY+Y97x4d5rdR75OD/+5dk3luQ6cyR35LkiWuo++VzrUcN6HnIBmNsLG5X38binbWHbCzOdYZsMGbQxmKmPeg/kulDmd/JdJjyDfO8pYfOfkuSv3iC+561sPaPJvkb28y/KMuu4fSDmb+KsGX+lyX5T0t63lLvGzNtpH9iTfVeuuV27CsDD8qCw7LmGk9J8oZMh/i/N8lbklySeS/CgrqvX9d4bqn72ExHcbw104nVfjLTIcrvT/JX11D73fPfyq8dW78zfZD5koW1D8zjfFum7xR9eJ5+QxZs12T6zv+2HzhnweFv8+//bFYOkV+Z/4Ikf7qw9itP8Lf4iCS/tob15KxMoe1/JPnfa1r3XrPldu48/0FJrlhY+/mZdiR8KtMRVR9I8sNJ7r+w7tdlOurkxkw7Qp40z9+f5EfLLr91LDA3N7eTv+X4DcZP5/gNxrMX1LWxeOLnOR02Fu9ycdtTrDtkY3GuPWSDMclfzvEbi4+c569jY/FRSf7G1vVvu42xHdZ+6i7Xfvom9Jzpe+N/qfpY79EyXNrzowf2/OiBfy9PynTiigdkOgP0/5vkGWuo+8TceYj5YzJ90La47h7VPpiVDwfXUPdrMp10aF09P2nEeGzp+cszfVC6rp7/yqhluOV5Xjv/u2j59S60ubmVumU+FHNT6m5Sz1s2Fs/48dj02kvqZvqk+UOZzr57NMkzV+5beiKBkbVfPKL2qLobPB6b2vMHBy7DUbVfmukDtasznezqikzfXX1nku9fY93/to66m1p71Djv8jLchJ4v33L7xSR/cOznJT33LrS5uZW6ZU3ff9ytuno2Hps4Hhl/ZuKNqr2JPRuPze95pfaIM3kPqbuptfW8az2/J4POIN97z74Au6q1dv2J7sr03bZSdUfW3sSeR9bexJ5H1h7Y81l9vl5O7/1oa+0pSf5Ta+1L59pLbGLtTex5ZG09717tz/be70jyh6213+q9/978PH/UWvtcwbqbWlvPu1P78ZlOZvf9Sf5B7/261tof9QXXdlwltMHuOzfJN2T6ns6qlunEC9Xqjqy9iT2PrL2JPY+sParuJ1trF/Ter0uS3vsftNb+ZqYLEn/FgrqbWnsTex5ZW8+7V/tPWmv37tMFmB9/bOZ8sfQlG8+j6m5qbT3vQu3e++eS/ERr7efnfz+ZNWYtoQ123y9lOtTkuq13tNbeUbDuyNqb2PPI2pvY88jao+o+L8lnV2f03j+b5HmttX+/oO6m1t7EnkfW1vPu1f5rvfc/nmuubizfM8nFBetuam09717t9N5vSfKtrbWDmQ6/XItjF30DAACgoLP2ugEAAABOTGgDAAAoTGgDAAAoTGgDAAAoTGgDAAAo7P8CDdpShFS81dsAAAAASUVORK5CYII=\n",
      "text/plain": [
       "<Figure size 1080x720 with 1 Axes>"
      ]
     },
     "metadata": {
      "needs_background": "light"
     },
     "output_type": "display_data"
    }
   ],
   "source": [
    "a = train_tmp.join(test_tmp).map(lambda row: (row[0], sum(row[1]))).collect()\n",
    "pd.Series([x[1] for x in a], index=[x[0] for x in a]).sort_index().plot.bar(figsize=(15, 10)) "
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "colab": {
     "base_uri": "https://localhost:8080/",
     "height": 493
    },
    "id": "nCySun0s-wQ5",
    "outputId": "6b7f38b3-58cf-4d46-c273-9e5ef2e33608"
   },
   "outputs": [
    {
     "data": {
      "text/plain": [
       "<matplotlib.axes._subplots.AxesSubplot at 0x7f1409777e80>"
      ]
     },
     "execution_count": 27,
     "metadata": {},
     "output_type": "execute_result"
    },
    {
     "data": {
      "image/png": "iVBORw0KGgoAAAANSUhEUgAAA20AAAJRCAYAAADWPV+oAAAABHNCSVQICAgIfAhkiAAAAAlwSFlzAAALEgAACxIB0t1+/AAAADh0RVh0U29mdHdhcmUAbWF0cGxvdGxpYiB2ZXJzaW9uMy4yLjIsIGh0dHA6Ly9tYXRwbG90bGliLm9yZy+WH4yJAAAgAElEQVR4nO3debhkd1kn8O8vNKCIQJY2RAI0YhBx1BhigiM8oCgGekbAQYRnRgIPTpwR3MatXWZwQ1tnXB+FeSJbUNkUkGiDgHHBLZCFGJaIidCRxEDCIuC4QfjNH+c0qb65Td9T99x7367+fJ6nnlv3VNV73/Orc6vOt86pc1rvPQAAANR0wk43AAAAwJEJbQAAAIUJbQAAAIUJbQAAAIUJbQAAAIUJbQAAAIXt2ukGkuSUU07pe/bs2ek2AAAAdsQVV1zxgd777vVuKxHa9uzZk8svv3yn2wAAANgRrbXrj3Sb3SMBAAAKE9oAAAAKE9oAAAAKE9oAAAAKE9oAAAAKE9oAAAAKE9oAAAAKE9oAAAAKE9oAAAAKE9oAAAAKE9oAAAAKE9oAAAAKE9oAAAAKE9oAAAAKE9oAAAAKE9oAAAAKE9oAAAAKE9oAAAAKE9oAAAAKE9oAAAAKE9oAAAAKE9oAAAAKE9oAAAAKE9oAAAAK27XTDbA69uw7cNT7HNy/dxs6AQCA1WFLGwAAQGFCGwAAQGFCGwAAQGFCGwAAQGFCGwAAQGFCGwAAQGFCGwAAQGFCGwAAQGFCGwAAQGFCGwAAQGFCGwAAQGFCGwAAQGFCGwAAQGFCGwAAQGFCGwAAQGFCGwAAQGFCGwAAQGFCGwAAQGFCGwAAQGFCGwAAQGFCGwAAQGG7droBdt6efQeOep+D+/duQycAAMBatrQBAAAUJrQBAAAUJrQBAAAUJrQBAAAUJrQBAAAUJrQBAAAUJrQBAAAUJrQBAAAUJrQBAAAUJrQBAAAUJrQBAAAUJrQBAAAUJrQBAAAUJrQBAAAUJrQBAAAUJrQBAAAUJrQBAAAUJrQBAAAUJrQBAAAUJrQBAAAUJrQBAAAUJrQBAAAUJrQBAAAUJrQBAAAUJrQBAAAUJrQBAAAUJrQBAAAUJrQBAAAUJrQBAAAUJrQBAAAUJrQBAAAUJrQBAAAUJrQBAAAUJrQBAAAUJrQBAAAUJrQBAAAUJrQBAAAUJrQBAAAUJrQBAAAUJrQBAAAUJrQBAAAUJrQBAAAUJrQBAAAUJrQBAAAUJrQBAAAUJrQBAAAUJrQBAAAUJrQBAAAUdtTQ1lq7d2vtj1pr72ytvaO19p3j9JNaa29srV07/jxxnN5aa7/cWruutXZ1a+2srZ4JAACAVbWRLW2fSPI9vfcHJXlIkme01h6UZF+SS3rvZyS5ZPw9SR6d5IzxckGS587eNQAAwHHiqKGt935T7/3K8frHklyT5F5JHpvkovFuFyV53Hj9sUle3AeXJrlHa+202TsHAAA4Dkz6TltrbU+SL0vy5iSn9t5vGm96X5JTx+v3SvLehYfdME4DAABgog2HttbaXZO8Msl39d4/unhb770n6VP+cGvtgtba5a21y2+55ZYpDwUAADhubCi0tdbumCGw/Wbv/VXj5Pcf2u1x/HnzOP3GJPdeePjp47TD9N4v7L2f3Xs/e/fu3cv2DwAAsNI2cvTIluT5Sa7pvf/8wk0XJzl/vH5+ktcsTH/KeBTJhyT5yMJulAAAAEywawP3+cok35zkba21q8ZpP5Rkf5JXtNaenuT6JE8cb3ttksckuS7JPyV52qwdAwAAHEeOGtp673+WpB3h5keuc/+e5Bmb7Ivj2J59B456n4P7925DJwAAsPMmHT0SAACA7bWR3SOZka1IAADAFLa0AQAAFCa0AQAAFCa0AQAAFCa0AQAAFCa0AQAAFCa0AQAAFOaQ/6wsp1cAAGAV2NIGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQ2K6dboDl7dl34Kj3Obh/7zZ0AgAAbBVb2gAAAAoT2gAAAAoT2gAAAAoT2gAAAAoT2gAAAAoT2gAAAAoT2gAAAAoT2gAAAAoT2gAAAAoT2gAAAAoT2gAAAAoT2gAAAAoT2gAAAAoT2gAAAAoT2gAAAAoT2gAAAAoT2gAAAAoT2gAAAAoT2gAAAAoT2gAAAAoT2gAAAArbtdMNwLFgz74DR73Pwf17t6ETAACON7a0AQAAFCa0AQAAFCa0AQAAFCa0AQAAFCa0AQAAFCa0AQAAFCa0AQAAFCa0AQAAFCa0AQAAFCa0AQAAFCa0AQAAFCa0AQAAFCa0AQAAFCa0AQAAFCa0AQAAFCa0AQAAFCa0AQAAFLZrpxuA48mefQeOep+D+/duQycAABwrbGkDAAAoTGgDAAAoTGgDAAAoTGgDAAAoTGgDAAAoTGgDAAAoTGgDAAAoTGgDAAAoTGgDAAAoTGgDAAAoTGgDAAAo7KihrbX2gtbaza21ty9M+9HW2o2ttavGy2MWbvvB1tp1rbV3tda+bqsaBwAAOB5sZEvbi5Kct870X+i9nzleXpskrbUHJXlSki8aH/Oc1tod5moWAADgeHPU0NZ7f1OSD22w3mOTvKz3/q+99/ckuS7JOZvoDwAA4Li2me+0PbO1dvW4++SJ47R7JXnvwn1uGKfdTmvtgtba5a21y2+55ZZNtAEAALC6lg1tz01y/yRnJrkpyc9NLdB7v7D3fnbv/ezdu3cv2QYAAMBqWyq09d7f33u/tff+ySS/ltt2gbwxyb0X7nr6OA0AAIAlLBXaWmunLfz6+CSHjix5cZIntdbu3Fq7X5Izkrxlcy0CAAAcv3Yd7Q6ttZcmeUSSU1prNyR5VpJHtNbOTNKTHEzyrUnSe39Ha+0VSd6Z5BNJntF7v3VrWgcAAFh9Rw1tvfcnrzP5+Z/m/s9O8uzNNAUAAMBgM0ePBAAAYIsJbQAAAIUJbQAAAIUJbQAAAIUJbQAAAIUJbQAAAIUJbQAAAIUJbQAAAIUJbQAAAIUJbQAAAIUJbQAAAIUJbQAAAIUJbQAAAIUJbQAAAIUJbQAAAIUJbQAAAIUJbQAAAIUJbQAAAIUJbQAAAIUJbQAAAIUJbQAAAIXt2ukGgOn27Dtw1Psc3L93GzoBAGCr2dIGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQ2K6dbuBYsWffgaPe5+D+vdvQCQAAcDyxpQ0AAKAwoQ0AAKAwoQ0AAKAwoQ0AAKAwoQ0AAKAwoQ0AAKAwoQ0AAKAwoQ0AAKAwoQ0AAKAwoQ0AAKCwXTvdALBz9uw7cNT7HNy/dxs6AQDgSGxpAwAAKExoAwAAKExoAwAAKExoAwAAKExoAwAAKExoAwAAKExoAwAAKExoAwAAKExoAwAAKExoAwAAKExoAwAAKExoAwAAKExoAwAAKExoAwAAKExoAwAAKExoAwAAKExoAwAAKExoAwAAKExoAwAAKExoAwAAKExoAwAAKExoAwAAKExoAwAAKExoAwAAKExoAwAAKExoAwAAKGzXTjcAHPv27Dtw1Psc3L93GzoBAFg9trQBAAAUJrQBAAAUJrQBAAAUJrQBAAAUJrQBAAAUJrQBAAAUJrQBAAAUdtTQ1lp7QWvt5tba2xemndRae2Nr7drx54nj9NZa++XW2nWttatba2dtZfMAAACrbiNb2l6U5Lw10/YluaT3fkaSS8bfk+TRSc4YLxckee48bQIAAByfjhraeu9vSvKhNZMfm+Si8fpFSR63MP3FfXBpknu01k6bq1kAAIDjzbLfaTu1937TeP19SU4dr98ryXsX7nfDOO12WmsXtNYub61dfssttyzZBgAAwGrb9IFIeu89SV/icRf23s/uvZ+9e/fuzbYBAACwkpYNbe8/tNvj+PPmcfqNSe69cL/Tx2kAAAAsYdnQdnGS88fr5yd5zcL0p4xHkXxIko8s7EYJAADARLuOdofW2kuTPCLJKa21G5I8K8n+JK9orT09yfVJnjje/bVJHpPkuiT/lORpW9AzAADAceOooa33/uQj3PTIde7bkzxjs00BAAAw2PSBSAAAANg6QhsAAEBhQhsAAEBhQhsAAEBhQhsAAEBhQhsAAEBhQhsAAEBhQhsAAEBhQhsAAEBhQhsAAEBhQhsAAEBhQhsAAEBhQhsAAEBhQhsAAEBhQhsAAEBhQhsAAEBhQhsAAEBhQhsAAEBhQhsAAEBhQhsAAEBhQhsAAEBhQhsAAEBhQhsAAEBhQhsAAEBhQhsAAEBhQhsAAEBhQhsAAEBhQhsAAEBhQhsAAEBhQhsAAEBhQhsAAEBhQhsAAEBhQhsAAEBhQhsAAEBhQhsAAEBhQhsAAEBhQhsAAEBhQhsAAEBhu3a6AYBD9uw7sKH7Hdy/d4s7AQCow5Y2AACAwoQ2AACAwoQ2AACAwoQ2AACAwoQ2AACAwoQ2AACAwoQ2AACAwoQ2AACAwlb+5NobOVmvE/UCAABV2dIGAABQmNAGAABQmNAGAABQmNAGAABQ2MofiAQ4PjkIEQCwKmxpAwAAKExoAwAAKExoAwAAKExoAwAAKExoAwAAKExoAwAAKExoAwAAKExoAwAAKExoAwAAKExoAwAAKExoAwAAKExoAwAAKExoAwAAKExoAwAAKExoAwAAKExoAwAAKExoAwAAKExoAwAAKExoAwAAKExoAwAAKExoAwAAKExoAwAAKExoAwAAKExoAwAAKExoAwAAKExoAwAAKExoAwAAKExoAwAAKExoAwAAKExoAwAAKExoAwAAKExoAwAAKExoAwAAKExoAwAAKExoAwAAKExoAwAAKExoAwAAKExoAwAAKExoAwAAKExoAwAAKExoAwAAKExoAwAAKExoAwAAKExoAwAAKExoAwAAKExoAwAAKExoAwAAKExoAwAAKGzXZh7cWjuY5GNJbk3yid772a21k5K8PMmeJAeTPLH3/uHNtQkAAHB8mmNL21f13s/svZ89/r4vySW99zOSXDL+DgAAwBK2YvfIxya5aLx+UZLHbcHfAAAAOC5sNrT1JG9orV3RWrtgnHZq7/2m8fr7kpy6yb8BAABw3NrUd9qSPLT3fmNr7XOSvLG19teLN/bee2utr/fAMeRdkCT3uc99NtkGAADAatrUlrbe+43jz5uTvDrJOUne31o7LUnGnzcf4bEX9t7P7r2fvXv37s20AQAAsLKWDm2ttc9qrX32oetJHpXk7UkuTnL+eLfzk7xms00CAAAcrzaze+SpSV7dWjtU5yW9999vrV2W5BWttacnuT7JEzffJgAAwPFp6dDWe393ki9dZ/oHkzxyM00BAAAw2IpD/gMAADAToQ0AAKAwoQ0AAKAwoQ0AAKAwoQ0AAKAwoQ0AAKCwzZynDWDl7dl34Kj3Obh/7zZ0AgAcr2xpAwAAKExoAwAAKExoAwAAKExoAwAAKExoAwAAKMzRIwGOIY5mCQDHH1vaAAAAChPaAAAACrN7JMA2sFsjALAsW9oAAAAKE9oAAAAKE9oAAAAK8502gOPQRr5jl/ieHQBUYEsbAABAYUIbAABAYUIbAABAYUIbAABAYUIbAABAYUIbAABAYUIbAABAYUIbAABAYUIbAABAYUIbAABAYUIbAABAYUIbAABAYbt2ugEAjm179h046n0O7t+7bXUAYNXY0gYAAFCY0AYAAFCY0AYAAFCY0AYAAFCY0AYAAFCY0AYAAFCYQ/4DsFKcggCAVWNLGwAAQGFCGwAAQGFCGwAAQGFCGwAAQGFCGwAAQGFCGwAAQGFCGwAAQGFCGwAAQGFCGwAAQGFCGwAAQGG7droBAFhle/Yd2ND9Du7fu8WdAHCssqUNAACgMKENAACgMKENAACgMKENAACgMKENAACgMKENAACgMKENAACgMKENAACgMCfXBoDjzEZO+O1k3wB12NIGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQ2K6dbgAA2Jg9+w4c9T4H9+/dhk5Wn7EGKrGlDQAAoDChDQAAoDChDQAAoDChDQAAoDAHIgEAluJgHUdnjIA52NIGAABQmNAGAABQmNAGAABQmNAGAABQmNAGAABQmKNHAgA7yhEWAT49W9oAAAAKE9oAAAAKs3skALASNrKbZWJXS+DYY0sbAABAYba0AQAwiYPHwPaypQ0AAKAwW9oAAIqzZQuOb7a0AQAAFFZ2S5tPlAAA5mX9Co5NtrQBAAAUVnZLGwAAUI8tttvPljYAAIDChDYAAIDC7B4JAAAju/5RkS1tAAAAhdnSBgDAMa/aFrJq/XBs27Itba2181pr72qtXdda27dVfwcAAGCVbcmWttbaHZL8apKvTXJDkstaaxf33t+5FX8PAIBjky1Sn96c41NtrOfqp1qduWslW7el7Zwk1/Xe3917/7ckL0vy2C36WwAAACur9d7nL9raE5Kc13v/lvH3b05ybu/9mQv3uSDJBeOvX5DkXUcpe0qSD8zQ3lx15qylzvbVUmf7aqmzfbXU2b5a6mxfLXW2r5Y621dLne2rdSzWuW/vffe6t/TeZ78keUKS5y38/s1JfmWTNS+fqbdZ6lTsaVXrVOxpVetU7GlV61TsaVXrVOxpVetU7GlV61TsaVXrVOxpVetU7KlKna3aPfLGJPde+P30cRoAAAATbFVouyzJGa21+7XW7pTkSUku3qK/BQAAsLK25OiRvfdPtNaemeT1Se6Q5AW993dssuyFm+9s1jpz1lJn+2qps3211Nm+WupsXy11tq+WOttXS53tq6XO9tVaqTpbciASAAAA5rFlJ9cGAABg84Q2AACAwoQ2AACAwoQ2AACAwrbk6JFzaK3dPcl5Se41Troxyet77/8wsU5Lcs6aOm/pE4/AMledsdZc81atzlxjPUs/sKpaa1+X5HE5/H/kNb33359QY1eSpyd5fJLPXayT5Pm9949vZ52x1qbna+Y6pcao2ljP2c9Y74FJHrump4t779dMrFNtOZprvkqNz5y1Co71SvYzZ61VXa6rvX8cVrPi0SNba09J8qwkb8htJ+U+PcnXJvmx3vuLN1jnUUmek+TaNXU+P8m39d7fsJ11xlpzzVu1OnON9Sz9LNQrs2KyBSs4ZV6gqo1RxbGecd5+MckDkrw4yQ3j5NOTPCXJtb3379xgnZcm+YckF62pc36Sk3rv37TNdeaar1nqjLWqjVG1sZ6ln7HWDyR5cpKXran1pCQv673v32CdUsvRjPNVanzmrFVwrFeynzlrrepyXfH94zC993KXJO9Kco91pp+Y5G8m1LkmyZ51pt8vyTXbXWfmeatWZ66xnqWf8TG/mOS1Gf75HzpenjRO+6UJdV6a5LlJHjL+w50+Xn9ukpdvd52Z520lx6joWM81b+v+HyRpGd5UNlXnaLdtd5255mtqnWNpjKqN9dR+Dt0/yR3XmX6nnXj+56wz13xVGp9VH+tV7KfqvFUao4rvH4uXqrtHtiTrbQL85HjbRu3Kbel20Y1J7rgDdZL55q1anbnGaK5+kuQxvfcH3O4PtPbyDC8UG/3E5MHr1LkhyaWttb+Z0M9cdZL55m1Vx6jiWM/V07+01r68937ZmulfnuRfJtT5UGvtG5O8svf+ySRprZ2Q5BuTfHgH6sw1X3PVSeqNUbWxnqufZHiN/9wk16+Zftp420ZVW47mmq9q4zNnrWpjvar9zFlrVZfriu8fn1I1tD07yZWttTckee847T4ZdpH7iQl1XpDkstbayxbq3DvDp+TP34E6yXzzVq3OXGM0Vz9JvRWTOf+Bq71AVRujimM9V09PTfLc1tpn57YPSu6d5CPjbRv1pCQ/k+Q5rbVDf/8eSf5ovG276zw188zXXHWSemNUbazn6idJvivJJa21a3P4a//nJ3nmhDpPTa3laK75qjY+c9aaq061MarWz5y1VnW5nqtOMu/rY5Ki32lLktbaiUm+Lrc/GMWkFa7W2hdm/S84vnMn6oy15pq3anXmGuu5+jkrw65n6/3zPaP3fsUG6+zJ8I/31blt5frQP96+3vt7trPOWGuueVvJMSo61rP1NNa7Zxb+R3rv75vy+DW1Tk6S3vsHl60xV5255mvO8RnrlRmjuepUW4bGDzHWHszqst77rUvUKrMczTVfFcdnzlqVxnpV+5mz1iov12XfP6qGNphLtRWTmeuUe4EqOEalxnrOnrZKa+2em32TmrNORdXGqNpYV+sHoIplXx+PufO0tdYunKnOj1aqM9aaa96q1fnRmeos1U/v/X299yvGy6ZWInrvH1xc0R5X5HeyzizztuJjVGqs5+xprdbalXPUyfTdvre0zlzzNeP4JMXGaK46BZehtNZ+b6Y6pZajGeer1PjMWavgWK9kP3PWWtXlusL7xzG3pa219uCN7o50lDr/sff+u1XqjLXmmrdqdeYa61n6GWtd2Xs/a4Y6B3rve6vUGWvNNW8rOUZFx3q2nmDVtNZO673ftNN9zG2u+VrV8ZlTtTGq1s+cLNdb55gLbQAkrbVTc/jumu9fokbL7b9L8JY+8Y1hrjpjrU3P18x1So1RtbGes5+FmiclSe/9Q5uoUWo5Gmtter7mqjPzfK3kWK94P6Xmba461ZbFuV8fS4a21trdk/xghpPZfk6GQ8DfnOEktPt77/+wwTrlTtQ747xVqzPXWM/Sz5qaZVZMZv8HLvQCVW2MKo71HD211s5M8n+T3D2Hn4D+HzKcyH5Du3C01h6V5DlJrl1T5/PHOm/Y5jpzzdcsdcZa1cao2ljP0s9Y6z5JfjbJI8c+WpK7JfnDDAfqObjBOqWWoxnnq9T4zFmr4FivZD9z1lrV5bri+8dh+hInd9vqS5LXJ/mBJPdcmHbPcdobJtSpeKLeueatWp25xnqWfsbHnZnk0gwn/v6D8fLX47SzJtR5VJLrkrwuyfPGy++P0x613XVmnreVHKOiYz3XvF2V5Nx1pj8kyV9NqHNNkj3rTL9fkmt2oM5c8zVLnaJjVG2sZ+lnfMxfJvmmJHdYmHaHDIfGvvRYXY5mnK9S47PiY72S/RSdt1JjNPNYz/b6+KnHLvOgrb4kedcyt61z31nORj5XnZnnrVqducZ6ln7G+5daMZnzH7jaC1S1MSo61nPN27Wf5rbrptRJsmud6XfaqTpzzdccdaqOUbWxnqOfDfR0xNuqL0fbNF/bPj7H8Vgfs/0cg/NW7TnbkfePxUvVk2tf31r7/iQX9XEXpHHXpKfmtpPvbUTFE/XONW/V6sw1RnP1kySf1Xt/89qJvfdLW2ufNaHOrtx2fq5FNya54w7USeabt1Udo4pjPVdPr2utHUjy4hx+IvunZNhyt1EvSHJZa+1la+o8KdOObDVXnbnma646ydaO0X0yfMJ8LI/1XP0kyRWtteckuWhNrfOTvHVCnWrL0VzzVW185qxVbaxXtZ85a63qcl3x/eNTqn6n7cQk+zKcqPnUDN9ren+Si5P8TN/glxTbbSez/aoM+6Mm85yot2XY33VSnbHWXPNWrc6ezDPWh/r5+rGfLNPPWOuXk9w/6//zvaf3/swN1vnBJE9Mst4/3it67z+9nXXGWnPN27EwRodWbo/1sZ6zp0dn/RPZv3ajNcY6X3iEOu+cWOdBGf5nN1vnMUeoM3W+ZhmfsdZcY1StzlxjPddzf6cM34u+3bxl+F70v06oNdf/x6brzDVfFcdnzlqVxnpV+5mz1iov1xXfPz5Vr2JoS5LW2gMzfC/q0t77Py5MP6/3vuG021o7N0MQ+dskD0zyFUneuczgj/VOHq/+Uu/9vyxTY029h2U4MMHb+rQvbZ+b5K977x9prd0lQ9A5K8k7kvxU7/0jG6zzHUle3XufuhVrbZ07JXlykr9PcmWS85J85djPhX3awVrun+QbMqzQ3prkXUle0nv/6BJ9reTK7Vir1ApuwZXS2V4sq63gMk1r7XN67zfvdB+LWmsn96InVwegoGX2qdzqS5LvyLCi/jtJDiZ57MJtV06o86wMBwu4PMlPJ7kkyf9M8qYkPzyhzsXrXP7x0PWJ8/aWhevfkmHz77OS/HmGLVIbrfOOjPvKJrkwyS8keehY61UT6nwkQ9D60yT/PckpSz5nv5nk5eOY/HqSVyX55iQvyrCr45Tn/g1JfiTJXyT51STPTvLOJI/Y6WXzeLgk+Zyd7mFNPyfvdA+VLhm28u/P8B25DyX54Hh9f5J7zPQ3XjfhvncbX19/PcmT19z2nAl17pnhwEW/muTkJD+a5Ookr0hy2oQ6J61zOZjkxCQnTRyH89aM+/PGnl6S5NQJdfYfem1N8uAk787wfYfrkzx8Qp0rx9fGz9vk8/vlGfaC+I0MH469McMeEpcl+bIJde6a5MfH96OPJLklw3vuU5foaVeSb81woJ6rx8vrkvy3JHecabm+cMJ97zD28xNJ/v2a235kQp27JPn+JN+X5DMy7PZ1cYYj5t11k/Mz6Tv142O+ZOH6Hcfl6eIkP5XkLhNrPXNhub5/hnWrDyd5c5IvnlDnVUn+8wzj8XkZdkn7iXHZ/LUkbwXJ+t0AAAocSURBVE/yW1nn+8Sfps4JSZ6W5PeS/NX4f/eyTFwHqbZMW66377nfwN/Z8HvsYY+bs4kZZ+Zth570JHsyhK7vHH9/68Q6dxgXro8muds4/TOTXD2hzpUZ3twekeTh48+bxusPnzhvb124flmS3eP1z8qwtW2jda5Z7G/NbVdN6WdcSB+VYR/bWzLst3t+ks+eUOfq8eeuDLsz3mH8vU0c67ctPPYuSf54vH6fKc/9+JiVXLkd719qBTcrunI71iq1gpsjH2F1X6Yd8fWsI1wenOSmCXVeOT5vj8vwJvnKJHc+9DxMqPP7Sb59nI+rx3m89zjtNRPqfDLJe9ZcPj7+fPfEsb5y4frzkvxkkvsm+e4kvzOhztsWrv9Rki8frz8gyeUT6rwnyf9J8ndJ3jL28blLLNNvSfLoDHtHvDfJE8bpj0zylxPqvCbD941PT/I/MnwoekaG76f81MSe5joC8XqvaSdleJ28YUKd52V4/fquJFck+fn1losN1HlFkp/LcOjvS5L8SpKHJfnfSX59Qp2PZViP+djC5dZD05dcpn8uwwerD8/wwe+LJz5n71i4fiDJ48frj0jy5xPq3JjktzO8T78iw6mD7rTEcv2mDB8+78sQ1r43w2vI05P84YQ6L8zwnvrQJL+Y4XX7azMcOfjbj9Vl2nK9fc/9WGuW99jDai7zoK2+LL4QjL/fNcMb+s9nYiBZ7/r4+5Q6J2R4c3xjkjPHaZPe/Bdq/VWGFeKTs+bNem2PR6nzW0metrCQnT1ef0CSyybUWRv47phh962XJrllQp23ZzgizonjP9tJ4/TPyLQj470tt63wnbg4RknePnGsV3Lldrx/qRXcrOjK7Vir1Apu5jvi660ZzoXzR+tc/nlCnavW/P7DGfYcOHnKcp3DX6//7tP9jaPU+Z7x/+OLF6a9Z8nn/soj9TCxp2ty254Rl665bcqHdYv9PCzDCtP7xufsgpnGesr70F+t+f2y8ecJGXbfnzLWcx2B+NYMH/YsvqYd+v3fJtS5euH6rgx7tLwqyZ0njtFV4882Pldt4fcpH2j+cobv1Z66MO09U8Z4nef+qoxbfKb2Mz7mXQvXL1tz25R5e+v4824Z9tB5bYYPtV6YaadDmWu5vnrN75eOP++caeszpZbptfNmud66537hedv0e+xhNZd50FZfxpk8c820XeMTe+uEOm/OuFk0yQkL0++eiSvJ4+NOzxCWfmXtC8KEGgcX/tnenXGrSIZgOmUl4O4ZPkn423E+Pz7W+5MkXzqhzhH/STNtk/J3j3//+gy7OF6SYdeEtyV51oQ635khhPxahnNhHQqmu5O8aeJYr+TK7drnbe2yOHE5mmUFNyu6cruBsd72FdwMuw9/fw5/kzs1Q3D/gwl13p7kjCPc9t6Jz9kJa6Y9NcMWxeuXGZ8kP7nscz/e/9Br9c8n+ews/yHbDRkC9veMr29t4bYpKwLfPj5vX53hU9xfyvAp8I9l2qfSt3udyLA3yXlJXjihzl9m2LviGzO8Zj9unP7wTPtw5C+SPHS8/vVJXr9w29RTtFw69rP4Xn1ChoMQvXlCnWuT3OcIt01Zrm/3P5nbvsow5VDkVy1cf8Ga26ae9+nBGd6LvmMcm8nL9bgcf0OS/5Q1K6FL9PPsDOshn5fkhzJsvblvxt3LJtRZb7k+OcNuhFO2kF2R4QO+c5J8ILd9mP35E/9fr0hy//H6WVlY98hwTIRjcpke77/Ky/XjN7tcz/Xcj/ef5T32sMct86CtvmR4w73nEW77ygl17nyE6adkwv7W6zx+bybu+rGBmndJcr8lHne3JF86LvQb3g1t4fEPmHEePjfj1owMR458QpJzlqjzReNjH7jJflZy5XZ8XKkV3Kzoyu34uFIruBm2QP9Mhg81Ppxhl6JrxmlTdml9QpIvOMJtj5tQ52eTfM0608/LtJWAH88634XIsML121OXyYVxvjTJ+5Z8/LPWXA7tzn7PTN+V7BEZvvf71gwfZr02yQWZ8N2WJC9bZj7WqfOlGfZEeF2GA3T9UoZdft+RNd9z2UCdt4zL4Z8dWp4yfMj2HRN72jOOz81J/ma83DxO2/B7Y5Jn5AgfXGbarm2/kYXdvhemf0uSj0+o87wjLNf3T/JnSzx3J2RYuf3TJH+/xONfuOZy6sIyfckS9Z6a4YPjD2TYy+adGb5HdPcJNSZ9IPtp6jwyw/EQrsmwe9srMwSem7NwbIQN1PnqDHtpXJvhA/Zzx+m7k/zsEsv0LePyfKiXHVmmx/uv6nL9ojmW64Xn/rrxuX/IMs/9+JhZ3mMPe9wyD3JxORYuOXzl9kM5fOX2xAl1Sq3cjo8ptYKb7Vm5vd1JKj9NjVlWbsdac63gfkkOX8F9wDh9mRXcByb5mrXLwHpvxhuo88gtrPPone4nw3eY/90ydbZpjHaqzhfOWGfTy+L4mHMzbCU5OcPRh783yWOWqHNObtu1+kEZPlCqVmdvFj7cWqLOw5L8ryX7OXeO+Vqnpy/K8MHdTo71uWv6WXYZ+oq5xmiscfJ4+Y1la6ypN+l9dTtqHaozdbleU+O0JB/cyfnKsEvlKZuts07dh47L0YZ3+V17KXvIf9hKrbWn9d5fuGp1NlurtfaZGXYNeHu1eatWZ6d6Gk/T8YwMH0CcmeEgTa8Zb7uy937WNtf59gxHkatSZ5b5qtjTzP18W4YPtHa8znj/Z2X47uiuDN8fPyfJH2c4CMDre+/PXrLOuRl2i161OnONz1J1trinlRij1trF60z+6gy7Aqb3/vVL1mkZzok7qc6ctbawTjLPGO1onbHWW3rv54zX/2uG94BXZ9hz53d77/s3WutT5kiPLi7H2iVLfiexep2KPa1qnZ3qKfMeXVedY6inVa2zUGuOoz2rsw11KvZUsM4sRx7PsNfJbEcwr9RTtTGae6wXri99tPjFy67AimqtXX2kmzJ8t+2YrFOxp1WtU7SnE3rv/5gkvfeDrbVHJPnt1tp9x1rqzFOnYk+rWidJPtF7vzXJP7XW/rb3/tGx7j+31j6pTrk6FXuqVufsDAdX++Ek39d7v6q19s+99z+ZUCMZjlkwR505a81Vp9oYzTnWJ7TWTszwfb3We78lSXrv/6+19okl6gltrLRTk3xdhu8QLWoZDgpxrNap2NOq1qnY0/tba2f23q9Kkt77P7bW/kOGk8p+sTqz1anY06rWSZJ/a63dpff+TxlWnJIkrbW7Zzg1iTq16lTsqVSd3vsnk/xCa+23xp/vzxLr3XPVqdjTqtYZ3T3D0Shbkt5aO633flNr7a6Z/qHWpxp0cVnJS4aThT/0CLe95FitU7GnVa1TsafMd3RddY6xnla1znj/WY72rM721KnYU7U66zx+liOPz1WnYk+rWmdNzaWOFt+7A5EAAACUdsJONwAAAMCRCW0AAACFCW0AAACFCW0AAACFCW0AAACF/X/vFLkG8kfY7AAAAABJRU5ErkJggg==\n",
      "text/plain": [
       "<Figure size 1080x720 with 1 Axes>"
      ]
     },
     "metadata": {
      "needs_background": "light"
     },
     "output_type": "display_data"
    }
   ],
   "source": [
    "a = train_tmp.cogroup(test_tmp).map(lambda row: (row[0], sum(list(row[1][0])) + sum(list(row[1][1])))).collect()\n",
    "pd.Series([x[1] for x in a], index=[x[0] for x in a]).sort_index().plot.bar(figsize=(15, 10)) "
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {
    "id": "FtJ2G-Eob6LI"
   },
   "source": [
    "## Сохранение\n"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "id": "sgRM7IV3b6LJ"
   },
   "outputs": [],
   "source": [
    "# Сохраните результат в csv sep=';' с header\n",
    "final_result = train_tmp.cogroup(test_tmp).map(lambda row: (row[0], sum(list(row[1][0])) + sum(list(row[1][1]))))"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "colab": {
     "base_uri": "https://localhost:8080/"
    },
    "id": "Nh82Aw06Bh2K",
    "outputId": "ad178ecd-a76b-4ea4-fe1a-a6d0f10290c3"
   },
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "rm: cannot remove 'result.csv': No such file or directory\n"
     ]
    }
   ],
   "source": [
    "!rm -r result.csv\n",
    "\n",
    "sc.parallelize(['key;value_ppi'])\\\n",
    ".union(final_result.map(lambda row: ';'.join(map(str, row)))).repartition(1).saveAsTextFile('result.csv')"
   ]
  }
 ],
 "metadata": {
  "colab": {
   "provenance": []
  },
  "kernelspec": {
   "display_name": "Python 3 (ipykernel)",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.9.15"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 1
}